# **BOUNDING BOX REGRESSION WITH RESNET 50**
**WE ARE GOING TO USE CALTECH-101 DATASET, I HAVE UPDATED THE DATASET FOR OUR PURPOSE, ZUERST IMPLEMENTIEREN DIE DATASET**

In [ ]:
#### ZUERST IMPLEMENTIREN WIR DIE BIBLIOTHEKEN, DI WIR BRAUCHEN KÖNNEN:
import os
import cv2
import imutils
import numpy as np
import matplotlib.pyplot as plt
import sys
import datetime
from sklearn.model_selection import train_test_split
import pandas as pd
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
import pickle
import zipfile # importing the 'zipfile' module

from google.colab import files
from google.colab import drive
import matplotlib.patches as patches

!pip install send2trash

from send2trash import send2trash
import pandas as pd

from keras.callbacks import EarlyStopping
import tensorflow as tf

from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt
import cv2

import cv2
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array

from keras.models import Sequential
## Import from keras_preprocessing not from keras.preprocessing
## from keras_preprocessing.image import ImageDataGenerator

from tensorflow.keras import optimizers

from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import time
import numpy as np
import tensorflow as tf
import tensorflow.compat.v1 as tf1
import tensorflow_datasets as tfds

import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

######################################################
############## RESNET50 ##############################
######################################################


import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanIoU

In [ ]:
# https://vijayabhaskar96.medium.com/tutorial-on-keras-flow-from-dataframe-1fd4493d237c
# https://studymachinelearning.com/keras-imagedatagenerator-with-flow_from_dataframe/
# https://stackoverflow.com/questions/66424141/imagedatagenerator-flow-from-dataframe-multi-output-regression-and-classificatio
# https://stackoverflow.com/questions/50781562/stratified-splitting-of-pandas-dataframe-into-training-validation-and-test-set

##################################################
########### DATA LOADER LINKS ####################
##################################################


In [ ]:
## WE ARE GOING TO IMPORT THE ZIP FILE, WHICH CONSIST THE IMAGES:

drive.mount('/content/drive')

In [ ]:
zip_file_name = "/content/drive/MyDrive/caltech-101_m2.zip"  # Zip dosyasının adını güncelleyin

with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall("caltech_files/")

In [ ]:
!pip install send2trash

from send2trash import send2trash

file_path = 'caltech-101_new/101_ObjectCategories/BACKGROUND_Google'

try:
    send2trash(file_path)
    print(f"File '{file_path}' has been moved to the trash.")
except OSError:
    print(f"Failed to delete file '{file_path}'.")

In [ ]:
## LET'S IMPORT THE DATAFRAMES FOR VALIDATION, TEST AND TRAIN

train_path = '/content/drive/My Drive/train_df.csv'
train_df = pd.read_csv(train_path)

val_path = '/content/drive/My Drive/val_df.csv'
val_df = pd.read_csv(val_path)

test_path = '/content/drive/My Drive/test_df.csv'
test_df = pd.read_csv(test_path)

In [ ]:
print(type(train_df))
print(type(test_df))
print(type(val_df))


print(len(train_df))
print(len(test_df))
print(len(val_df))


In [ ]:
train_df.to_csv('/content/train_df.csv', index=False)
val_df.to_csv('/content/val_df.csv', index=False)
test_df.to_csv('/content/test_df.csv', index=False)


In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
val_df.head()

In [ ]:

def plot_eda(dataframe):
    # Count the number of unique classes
    num_classes = len(dataframe['Class Name'].unique())

    # Count the number of images
    num_images = dataframe.shape[0]

    # Count the number of images per class
    images_per_class = dataframe['Class Name'].value_counts()

    # Plotting
    plt.figure(figsize=(12, 12))
    plt.barh(range(num_classes), images_per_class.values)
    plt.yticks(range(num_classes), images_per_class.index)
    plt.xlabel('Number of Images')
    plt.ylabel('Class')
    plt.title('Number of Images per Class')

    # Display the number of images
    for i, v in enumerate(images_per_class.values):
        plt.text(v, i, str(v), color='black', va='center')

    # Adjust the layout and display the plot
    plt.tight_layout()
    plt.show()


In [ ]:
# Call the function for each dataframe
plot_eda(train_df)
plot_eda(val_df)
plot_eda(test_df)

In [ ]:
# Combine the dataframes
combined_df = pd.concat([train_df, val_df, test_df], ignore_index=True)

def plot_combined_eda(dataframe):
    # Count the number of unique classes
    num_classes = len(dataframe['Class Name'].unique())

    # Count the number of images
    num_images = dataframe.shape[0]

    # Count the number of images per class
    images_per_class = dataframe['Class Name'].value_counts()

    # Plotting
    plt.figure(figsize=(12, 12))
    plt.barh(range(num_classes), images_per_class.values, height=0.5)

    # Adjust the spacing between the y-tick labels
    plt.gca().set_yticks(range(num_classes))
    plt.gca().set_yticklabels(images_per_class.index, ha='right')

    plt.xlabel('Number of Images')
    plt.ylabel('Class')
    plt.title('Number of Images per Class (Combined)')

    # Display the number of images
    for i, v in enumerate(images_per_class.values):
        plt.text(v + 5, i, str(v), color='black', va='center')

    # Display the total number of images
    plt.text(0, num_classes+1, f'Total Images: {num_images}', fontsize=12, fontweight='bold')

    # Adjust the layout and display the plot
    plt.tight_layout()
    plt.show()

# Call the function for the combined dataframe
plot_combined_eda(combined_df)

### **WE HAVE THE DATAFRAMES FOR TRAINING, VALIDATION AND TEST LET'S CONTINUE WITH TRAINING THE MODEL**

In [ ]:
train_df = pd.read_csv('/content/drive/My Drive/train_df.csv')
val_df = pd.read_csv('/content/drive/My Drive/val_df.csv')
test_df = pd.read_csv('/content/drive/My Drive/test_df.csv')

In [ ]:
column_names = ['Image Path', 'Class Name', 'Image Name', 'Bounding Box', 'Height', 'Width', 'label',
                'normalized', 'Xmin', 'Ymin', 'Xmax', 'Ymax']


carside_df.columns = column_names
val_carside_df.columns = column_names
test_carside_df.columns = column_names

In [ ]:
def load_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))
    image = image.astype(np.float32) / 255.0
    return image

In [ ]:

def plot_images_with_bboxes(df):
    fig, axes = plt.subplots(1, 5, figsize=(20, 4))
    for i, row in df.head(5).iterrows():
        image_path = row['Image Path']
        class_name = row['Class Name']
        image_name = row['Image Name']
        xmin, ymin, xmax, ymax = row['Xmin']*224, row['Ymin']*224, row['Xmax']*224, row['Ymax']*224

        image = load_image(image_path)

        ax = axes[i]
        ax.imshow(image)
        ax.axis('off')

        # Add bounding box
        bbox = patches.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(bbox)

        ax.set_title(f'Class: {class_name}\nImage: {image_name}')

    plt.show()

# Plot images with bounding boxes for val_df, test_df, train_df
plot_images_with_bboxes(val_carside_df)
plot_images_with_bboxes(test_carside_df)
plot_images_with_bboxes(carside_df)

In [ ]:
####################################################
############## MOST IMPORTANT PART #################
####################################################



def resize_image(image, target_size):
    return cv2.resize(image, target_size)

def plot_images(images, boxes, labels, dataset_name):
    fig, axes = plt.subplots(1, len(images), figsize=(12, 4))
    for i, ax in enumerate(axes):
        ax.imshow(images[i])
        xmin, ymin, xmax, ymax = boxes[i]
        ax.add_patch(plt.Rectangle((xmin*224, ymin*224), xmax*224 - xmin*224, ymax*224 - ymin*224, fill=False, color='red', linewidth=2))
        ax.set_title(labels[i])
        ax.axis('off')
    plt.suptitle(f'{dataset_name} Images with Bounding Boxes')
    plt.show()

# Append data from val_df into arrays
val_images = []
val_boxes = []
val_labels = []

for index, row in val_carside_df.iterrows():
    image_path = row['Image Path']
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = resize_image(image, (224, 224))
    val_images.append(image)

    xmin, ymin, xmax, ymax = map(float, [row['Xmin'], row['Ymin'], row['Xmax'], row['Ymax']])
    val_boxes.append([xmin, ymin, xmax, ymax])

    label = row['Class Name']
    val_labels.append(label)

# Convert the lists to NumPy arrays
val_images = np.array(val_images, dtype='float32')
val_boxes = np.array(val_boxes, dtype='float32')
val_labels = np.array(val_labels)

# Normalize the images
val_images = val_images / 255.0

# Plot the images, bounding boxes, and class names for the first 4 images from the val_df dataset
plot_images(val_images[:5], val_boxes[:5], val_labels[:5], 'Validation')

# Append data from train_df into arrays
train_images = []
train_boxes = []
train_labels = []

for index, row in carside_df.iterrows():
    image_path = row['Image Path']
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = resize_image(image, (224, 224))
    train_images.append(image)

    xmin, ymin, xmax, ymax = map(float, [row['Xmin'], row['Ymin'], row['Xmax'], row['Ymax']])
    train_boxes.append([xmin, ymin, xmax, ymax])

    label = row['Class Name']
    train_labels.append(label)

# Convert the lists to NumPy arrays
train_images = np.array(train_images, dtype='float32')
train_boxes = np.array(train_boxes, dtype='float32')
train_labels = np.array(train_labels)

# Normalize the images
train_images = train_images / 255.0

# Plot the images, bounding boxes, and class names for the first 4 images from the train_df dataset
plot_images(train_images[:5], train_boxes[:5], train_labels[:5], 'Train')

# Append data from test_df into arrays
test_images = []
test_boxes = []
test_labels = []

for index, row in test_carside_df.iterrows():
    image_path = row['Image Path']
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = resize_image(image, (224, 224))
    test_images.append(image)

    xmin, ymin, xmax, ymax = map(float, [row['Xmin'], row['Ymin'], row['Xmax'], row['Ymax']])
    test_boxes.append([xmin, ymin, xmax, ymax])

    label = row['Class Name']
    test_labels.append(label)

# Convert the lists to NumPy arrays
test_images = np.array(test_images, dtype='float32')
test_boxes = np.array(test_boxes, dtype='float32')
test_labels = np.array(test_labels)

# Normalize the images
test_images = test_images / 255.0

# Plot the images, bounding boxes, and class names for the first 4 images from the test_df dataset
plot_images(test_images[:5], test_boxes[:5], test_labels[:5], 'Test')


In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Perform label encoding on the target labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
val_labels_encoded = label_encoder.transform(val_labels)

# Convert the encoded labels to one-hot encoding
train_labels_onehot = to_categorical(train_labels_encoded, num_classes=101)
val_labels_onehot = to_categorical(val_labels_encoded, num_classes=101)


# Model architecture
input_layer = Input(shape=(224, 224, 3))
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=input_layer)

flatten = Flatten()(base_model.output)

bboxHead = Dense(1024, activation="relu")(flatten)



bboxHead = Dense(512, activation="relu")(bboxHead)


bboxHead = Dense(256, activation="relu")(bboxHead)



bboxHead = Dense(128, activation="relu")(bboxHead)



bboxHead = Dense(64, activation="relu")(bboxHead)


bboxHead = Dense(32, activation="relu")(bboxHead)

bbox_output = Dense(4, activation='linear', name='bounding_box')(bboxHead)



softmaxHead = Dense(1024, activation="relu")(flatten)
softmaxHead = Dropout(0.5)(softmaxHead)


softmaxHead = Dense(512, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)

softmaxHead = Dense(256, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)

softmaxHead = Dense(128, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)

class_output = Dense(101, activation='softmax', name='class_output')(softmaxHead)

model = Model(inputs=input_layer, outputs=[class_output, bbox_output])

# Loss function
losses = {
    'class_output': 'categorical_crossentropy',
    'bounding_box': 'mean_squared_error'
}

# Compile the model
model.compile(optimizer='adam', loss=losses)


model.summary()

In [ ]:
from keras.callbacks import EarlyStopping

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=7)


# Train the model with early stopping
history = model.fit(train_images, {'class_output': train_labels_onehot, 'bounding_box': train_boxes},
                    validation_data=(val_images, {'class_output': val_labels_onehot, 'bounding_box': val_boxes}),
                    epochs=50, batch_size=32)

# Save the trained model
model.save('/content/drive/MyDrive/trained_model.h5')

In [ ]:
import pickle

# Save the label encoder
with open('/content/label_encoder.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)

In [ ]:
# Plot the losses
plt.figure(figsize=(12, 6))
plt.style.use('seaborn')

# Classification Loss
plt.subplot(1, 2, 1)
plt.plot(history.history['class_output_loss'], label='Train Classification Loss', color='blue')
plt.plot(history.history['val_class_output_loss'], label='Validation Classification Loss', color='orange')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Classification Loss', fontsize=14)
plt.legend()

# Bounding Box Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['bounding_box_loss'], label='Train Bounding Box Loss', color='blue')
plt.plot(history.history['val_bounding_box_loss'], label='Validation Bounding Box Loss', color='orange')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Bounding Box Loss', fontsize=14)
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Find the best and worst validation losses
best_val_class_loss = min(history.history['val_class_output_loss'])
worst_val_class_loss = max(history.history['val_class_output_loss'])
best_val_bbox_loss = min(history.history['val_bounding_box_loss'])
worst_val_bbox_loss = max(history.history['val_bounding_box_loss'])

# Plot the losses
plt.figure(figsize=(12, 6))
plt.style.use('seaborn')

# Classification Loss
plt.subplot(1, 2, 1)
plt.plot(history.history['class_output_loss'], label='Train Classification Loss', color='blue')
plt.plot(history.history['val_class_output_loss'], label='Validation Classification Loss', color='orange')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Classification Loss', fontsize=14)
plt.legend()

# Bounding Box Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['bounding_box_loss'], label='Train Bounding Box Loss', color='blue')
plt.plot(history.history['val_bounding_box_loss'], label='Validation Bounding Box Loss', color='orange')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Bounding Box Loss', fontsize=14)
plt.legend()

# Display the best and worst values
plt.text(len(history.history['val_class_output_loss']) // 2, best_val_class_loss, f'Best: {best_val_class_loss:.4f}', color='green', ha='center')
plt.text(len(history.history['val_class_output_loss']) // 2, worst_val_class_loss, f'Worst: {worst_val_class_loss:.4f}', color='red', ha='center')
plt.text(len(history.history['val_bounding_box_loss']) // 2, best_val_bbox_loss, f'Best: {best_val_bbox_loss:.4f}', color='green', ha='center')
plt.text(len(history.history['val_bounding_box_loss']) // 2, worst_val_bbox_loss, f'Worst: {worst_val_bbox_loss:.4f}', color='red', ha='center')

plt.tight_layout()
plt.show()


In [ ]:
# Find the best and worst validation losses
best_val_class_loss = min(history.history['val_class_output_loss'])
worst_val_class_loss = max(history.history['val_class_output_loss'])
best_val_bbox_loss = min(history.history['val_bounding_box_loss'])
worst_val_bbox_loss = max(history.history['val_bounding_box_loss'])

# Plot the Classification Loss
plt.figure(figsize=(8, 6))
plt.plot(history.history['class_output_loss'], label='Train Classification Loss', color='blue')
plt.plot(history.history['val_class_output_loss'], label='Validation Classification Loss', color='orange')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Classification Loss', fontsize=14)
plt.legend()
plt.tight_layout()
plt.show()

# Display the best and worst values for Classification Loss
print(f'Best Validation Classification Loss: {best_val_class_loss:.4f}')
print(f'Worst Validation Classification Loss: {worst_val_class_loss:.4f}')
print()

# Plot the Bounding Box Loss
plt.figure(figsize=(8, 6))
plt.plot(history.history['bounding_box_loss'], label='Train Bounding Box Loss', color='blue')
plt.plot(history.history['val_bounding_box_loss'], label='Validation Bounding Box Loss', color='orange')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Bounding Box Loss', fontsize=14)
plt.legend()
plt.tight_layout()
plt.show()

# Display the best and worst values for Bounding Box Loss
print(f'Best Validation Bounding Box Loss: {best_val_bbox_loss:.4f}')
print(f'Worst Validation Bounding Box Loss: {worst_val_bbox_loss:.4f}')


In [ ]:
# Find the best and worst losses
best_train_class_loss = min(history.history['class_output_loss'])
worst_train_class_loss = max(history.history['class_output_loss'])
best_train_bbox_loss = min(history.history['bounding_box_loss'])
worst_train_bbox_loss = max(history.history['bounding_box_loss'])

# Find the best and worst validation losses
best_val_class_loss = min(history.history['val_class_output_loss'])
worst_val_class_loss = max(history.history['val_class_output_loss'])
best_val_bbox_loss = min(history.history['val_bounding_box_loss'])
worst_val_bbox_loss = max(history.history['val_bounding_box_loss'])

# Plot the Classification Loss
plt.figure(figsize=(8, 6))
plt.plot(history.history['class_output_loss'], label='Train Classification Loss', color='blue')
plt.plot(history.history['val_class_output_loss'], label='Validation Classification Loss', color='orange')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Classification Loss', fontsize=14)
plt.legend()
plt.tight_layout()
plt.show()

# Display the best and worst values for Classification Loss
print(f'Best Train Classification Loss: {best_train_class_loss:.4f}')
print(f'Worst Train Classification Loss: {worst_train_class_loss:.4f}')
print(f'Best Validation Classification Loss: {best_val_class_loss:.4f}')
print(f'Worst Validation Classification Loss: {worst_val_class_loss:.4f}')
print()

# Plot the Bounding Box Loss
plt.figure(figsize=(8, 6))
plt.plot(history.history['bounding_box_loss'], label='Train Bounding Box Loss', color='blue')
plt.plot(history.history['val_bounding_box_loss'], label='Validation Bounding Box Loss', color='orange')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Bounding Box Loss', fontsize=14)
plt.legend()
plt.tight_layout()
plt.show()

# Display the best and worst values for Bounding Box Loss
print(f'Best Train Bounding Box Loss: {best_train_bbox_loss:.4f}')
print(f'Worst Train Bounding Box Loss: {worst_train_bbox_loss:.4f}')
print(f'Best Validation Bounding Box Loss: {best_val_bbox_loss:.4f}')
print(f'Worst Validation Bounding Box Loss: {worst_val_bbox_loss:.4f}')


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Model architecture
input_layer = Input(shape=(224, 224, 3))
base_model = ResNet50(weights=None, include_top=False, input_tensor=input_layer)


In [ ]:
flatten = Flatten()(base_model.output)

bboxHead = Dense(1024, activation="relu")(flatten)
bboxHead = Dense(512, activation="relu")(bboxHead)
bboxHead = Dense(256, activation="relu")(bboxHead)
bboxHead = Dense(128, activation="relu")(bboxHead)
bboxHead = Dense(64, activation="relu")(bboxHead)
bboxHead = Dense(32, activation="relu")(bboxHead)

bbox_output = Dense(4, activation='linear', name='bounding_box')(bboxHead)

model = Model(inputs=input_layer, outputs=bbox_output)

# Load the trained model weights
model.load_weights('/content/drive/MyDrive/project_mark1/model_top_layers.h5')

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.callbacks import EarlyStopping

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Model architecture
input_layer = Input(shape=(224, 224, 3))
base_model = ResNet50(weights=None, include_top=False, input_tensor=input_layer)

flatten = Flatten()(base_model.output)

bboxHead = Dense(1024, activation="relu")(flatten)
bboxHead = Dense(512, activation="relu")(bboxHead)
bboxHead = Dense(256, activation="relu")(bboxHead)
bboxHead = Dense(128, activation="relu")(bboxHead)
bboxHead = Dense(64, activation="relu")(bboxHead)
bboxHead = Dense(32, activation="relu")(bboxHead)

bbox_output = Dense(4, activation='linear', name='bounding_box')(bboxHead)

model = Model(inputs=input_layer, outputs=bbox_output)

# Load the weights of the top layers
model.load_weights('/content/drive/MyDrive/project_mark1/model_top_layers.h5', by_name=True)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Define early stopping callback
callback = EarlyStopping(monitor='loss', patience=7)

# Train the model with early stopping
history = model.fit(train_images, train_boxes,
                    validation_data=(val_images, val_boxes),
                    epochs=50, batch_size=32, callbacks=[callback])

# Save the trained model
model.save('/content/drive/MyDrive/trained_bbox_model.h5')


In [ ]:
# Get the best and worst values of val_loss and loss
best_val_loss = min(loss_callback.val_losses)
worst_val_loss = max(loss_callback.val_losses)
best_loss = min(loss_callback.losses)
worst_loss = max(loss_callback.losses)

# Plot the best and worst values
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(loss_callback.val_losses)
plt.xlabel('Epochs')
plt.ylabel('Validation Loss')
plt.title(f'Best: {best_val_loss:.4f}\nWorst: {worst_val_loss:.4f}')

plt.subplot(1, 2, 2)
plt.plot(loss_callback.losses)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title(f'Best: {best_loss:.4f}\nWorst: {worst_loss:.4f}')

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.patches as patches

# Define a function to plot images with bounding boxes
def plot_images(images, boxes, labels, dataset_name):
    fig, axes = plt.subplots(1, len(images), figsize=(12, 4))
    for i, ax in enumerate(axes):
        ax.imshow(images[i])
        xmin, ymin, xmax, ymax = boxes[i]
        ax.add_patch(plt.Rectangle((xmin*224, ymin*224), (xmax-xmin)*224, (ymax-ymin)*224, fill=False, color='red', linewidth=2))
        ax.set_title(labels[i])
        ax.axis('off')
    plt.suptitle(f'{dataset_name} Images with Bounding Boxes')
    plt.show()

# Get the predictions on the test data
predictions = model.predict(test_images)

# Calculate the IoU scores for each predicted bounding box
iou_scores = []
for i in range(len(predictions)):
    pred_box = predictions[i][:4] * np.array([190, 300, 190, 300])
    true_box = test_boxes[i]
    iou = calculate_iou(pred_box, true_box)
    iou_scores.append(iou)

# Sort the images based on the IoU scores (best and worst)
best_indices = np.argsort(iou_scores)[-5:]  # Get indices of images with highest IoU scores
worst_indices = np.argsort(iou_scores)[:5]  # Get indices of images with lowest IoU scores

# Plot the images with the best bounding box predictions
best_images = test_images[best_indices]
best_boxes = predictions[best_indices][:, :4] * np.array([190, 300, 190, 300])
best_labels = [test_labels[i] for i in best_indices]
plot_images(best_images, best_boxes, best_labels, 'Best Predictions')

# Plot the images with the worst bounding box predictions
worst_images = test_images[worst_indices]
worst_boxes = predictions[worst_indices][:, :4] * np.array([190, 300, 190, 300])
worst_labels = [test_labels[i] for i in worst_indices]
plot_images(worst_images, worst_boxes, worst_labels, 'Worst Predictions')


In [ ]:
# Calculate the IoU scores for each predicted bounding box
iou_scores = []
for i in range(len(predictions)):
    pred_box = predictions[i][4:8] * np.array([190, 300, 190, 300])
    true_box = test_boxes[i]
    iou = calculate_iou(pred_box, true_box)
    iou_scores.append(iou)

# Sort the images based on the IoU scores (best and worst)
best_indices = np.argsort(iou_scores)[-5:]  # Get indices of images with highest IoU scores
worst_indices = np.argsort(iou_scores)[:5]  # Get indices of images with lowest IoU scores

# Plot the images with the best bounding box predictions
best_images = test_images[best_indices]
best_boxes = predictions[best_indices][:, 4:8] * np.array([190, 300, 190, 300])
best_labels = [test_labels[i] for i in best_indices]
plot_images(best_images, best_boxes, best_labels, 'Best Predictions')

# Plot the images with the worst bounding box predictions
worst_images = test_images[worst_indices]
worst_boxes = predictions[worst_indices][:, 4:8] * np.array([190, 300, 190, 300])
worst_labels = [test_labels[i] for i in worst_indices]
plot_images(worst_images, worst_boxes, worst_labels, 'Worst Predictions')


In [ ]:
# Get the best and worst values of val_loss and loss
best_val_loss = min(loss_callback.val_losses)
worst_val_loss = max(loss_callback.val_losses)
best_loss = min(loss_callback.losses)
worst_loss = max(loss_callback.losses)

# Plot the best and worst values
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(loss_callback.val_losses)
plt.xlabel('Epochs')
plt.ylabel('Validation Loss')
plt.title(f'Best: {best_val_loss:.4f}\nWorst: {worst_val_loss:.4f}')

plt.subplot(1, 2, 2)
plt.plot(loss_callback.losses)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title(f'Best: {best_loss:.4f}\nWorst: {worst_loss:.4f}')

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Perform label encoding on the target labels
test_labels_encoded = label_encoder.transform(test_labels)

# Convert the encoded labels to one-hot encoding
test_labels_onehot = to_categorical(test_labels_encoded, num_classes=100)

# Evaluate the model on the test data
test_loss, test_class_output_loss, test_bbox_output_loss = model.evaluate(test_images, {'class_output': test_labels_onehot, 'bounding_box': test_boxes})

print('Test Loss:', test_loss)
print('Test Classification Loss:', test_class_output_loss)
print('Test Bounding Box Loss:', test_bbox_output_loss)

In [ ]:
import matplotlib.patches as patches
from sklearn.metrics import jaccard_score

# Define a function to calculate the IoU score
def calculate_iou(box1, box2):
    # Extract coordinates from the bounding boxes
    xmin1, ymin1, xmax1, ymax1 = box1
    xmin2, ymin2, xmax2, ymax2 = box2

    # Calculate the coordinates of the intersection rectangle
    xmin_inter = max(xmin1, xmin2)
    ymin_inter = max(ymin1, ymin2)
    xmax_inter = min(xmax1, xmax2)
    ymax_inter = min(ymax1, ymax2)

    # Calculate the area of intersection rectangle
    inter_area = max(0, xmax_inter - xmin_inter + 1) * max(0, ymax_inter - ymin_inter + 1)

    # Calculate the areas of the bounding boxes
    box1_area = (xmax1 - xmin1 + 1) * (ymax1 - ymin1 + 1)
    box2_area = (xmax2 - xmin2 + 1) * (ymax2 - ymin2 + 1)

    # Calculate the IoU score
    iou = inter_area / float(box1_area + box2_area - inter_area)
    return iou

# Define a function to plot test images with predicted bounding box, original bounding box, predicted class, and original class
def plot_test_images(test_images, test_boxes, test_labels, pred_boxes, pred_labels, step=10):
    num_plots = min(len(test_images), 10)
    plt.figure(figsize=(15, 15))
    for i in range(0, len(test_images), step):
        if i // step >= num_plots:
            break
        plt.subplot(5, 2, (i // step) + 1)

        # Denormalize the bounding box coordinates
        x1, y1, x2, y2 = test_boxes[i] * 224
        xmin = int(x1)
        ymin = int(y1)
        xmax = int(x2)
        ymax = int(y2)

        # Denormalize the image array
        image = test_images[i] * 255
        image = image.astype(np.uint8)

        plt.imshow(image)
        plt.axis('off')
        plt.title(f"Original Class: {test_labels[i]}\nPredicted Class: {pred_labels[i]}")

        # Plot original bounding box
        rect = patches.Rectangle((xmin, ymin), (xmax - xmin), (ymax - ymin), linewidth=2, edgecolor='r', facecolor='none')
        plt.gca().add_patch(rect)

        # Denormalize the predicted bounding box coordinates
        x1, y1, x2, y2 = pred_boxes[i] * 224
        xmin = int(x1)
        ymin = int(y1)
        xmax = int(x2)
        ymax = int(y2)

        # Plot predicted bounding box
        rect = patches.Rectangle((xmin, ymin), (xmax - xmin), (ymax - ymin), linewidth=2, edgecolor='g', facecolor='none')
        plt.gca().add_patch(rect)

        # Calculate IoU score
        iou_score = calculate_iou(test_boxes[i] * 224, pred_boxes[i] * 224)
        plt.text(xmin, ymin, f"IoU: {iou_score:.2f}", color='b', fontsize=8, backgroundcolor='w')

    plt.tight_layout()
    plt.show()

# Perform predictions on the test data
preds = model.predict(test_images)
pred_labels = label_encoder.inverse_transform(np.argmax(preds[0], axis=1))
pred_boxes = preds[1]

# Plot every 10th test image with predicted bounding box, original bounding box, predicted class, original class, and IoU score
plot_test_images(test_images, test_boxes, test_labels, pred_boxes, pred_labels, step=1)


In [ ]:
from sklearn.metrics import precision_recall_fscore_support
import matplotlib.pyplot as plt
import numpy as np

# Calculate precision, recall, and F1 scores for each class
precision, recall, f1, _ = precision_recall_fscore_support(test_labels, pred_labels, average=None)

# Get the unique class labels
classes = np.unique(test_labels)

# Set the width of the bars
bar_width = 0.35

# Set the x locations of the bars
index = np.arange(len(classes))

# Set the figure size
plt.figure(figsize=(12, 12))

# Plot the recall scores
plt.bar(index, recall, bar_width, label='Recall', color='skyblue')
plt.xlabel('Class')
plt.ylabel('Score')
plt.title('Recall Scores by Class')
plt.xticks(index, classes, rotation='vertical')

# Add scores as text inside each bar
for i, v in enumerate(recall):
    plt.text(i, v, f"{v:.2f}", color='black', ha='center', va='bottom', fontsize=8)

plt.legend()
plt.tight_layout()
plt.show()

# Set the figure size
plt.figure(figsize=(10, 6))

# Plot the F1 scores
plt.bar(index, f1, bar_width, label='F1 Score', color='lightgreen')
plt.xlabel('Class')
plt.ylabel('Score')
plt.title('F1 Scores by Class')
plt.xticks(index, classes, rotation='vertical')

# Add scores as text inside each bar
for i, v in enumerate(f1):
    plt.text(i, v, f"{v:.2f}", color='black', ha='center', va='bottom', fontsize=8)

plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
class_iou_scores = []

# Calculate the mean IoU score for each class
for class_label in classes:
    # Get the indices of samples belonging to the current class
    class_indices = np.where(test_labels == class_label)[0]

    # Extract the predicted and ground truth bounding boxes for the current class
    class_pred_boxes = pred_boxes[class_indices]
    class_test_boxes = test_boxes[class_indices]

    # Calculate the IoU score for each sample
    iou_scores = []

    for i in range(len(class_indices)):
        iou_score = calculate_iou(class_test_boxes[i], class_pred_boxes[i])
        iou_scores.append(iou_score)

    # Calculate the mean IoU score for the current class
    mean_iou_score = np.mean(iou_scores)

    # Append the class label and mean IoU score to the list of class IoU scores
    class_iou_scores.append((class_label, mean_iou_score))

# Sort the mean IoU scores from best to worst
class_iou_scores.sort(key=lambda x: x[1], reverse=True)

# Extract the class labels and mean IoU scores into separate lists
sorted_classes = [x[0] for x in class_iou_scores]
sorted_scores = [x[1] for x in class_iou_scores]

# Calculate the total mean IoU score
total_mean_iou = np.mean(sorted_scores)

# Plot the mean IoU scores for each class
fig, ax = plt.subplots(figsize=(12, 12))
bars = ax.barh(range(len(sorted_classes)), sorted_scores, color='purple')
ax.set_xlabel('Mean IoU Score')
ax.set_ylabel('Class')
ax.set_title('Mean IoU Scores by Class')
ax.set_yticks(range(len(sorted_classes)))
ax.set_yticklabels(sorted_classes)
ax.invert_yaxis()

# Add scores as text inside each bar
for bar, score in zip(bars, sorted_scores):
    ax.text(bar.get_width() + 0.005, bar.get_y() + bar.get_height() / 2, f"{score:.2f}",
            va='center', fontsize=7)

# Add total mean IoU score as text at the top
ax.text(0, -1, f"Total Mean IoU: {total_mean_iou:.2f}", ha='left', va='center', fontsize=12)

# Add a vertical line at x=0.5 for reference
ax.axvline(x=0.5, color='red', linestyle='--', linewidth=1)

plt.tight_layout()
plt.show()

In [ ]:
from tensorflow.keras.utils import plot_model

# Assuming you have already loaded your model
model = load_model('/content/drive/MyDrive/project_mark1/model_top_layers.h5')

# Plot the model architecture
plot_model(model, to_file='model_architecture.png', show_shapes=True)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Get the number of images for each class from test_df
class_counts = test_df["Class Name"].value_counts()

# Set the figure size
plt.figure(figsize=(10, 6))

# Plot the number of images per class
class_counts.plot(kind='bar')
plt.xlabel('Class')
plt.ylabel('Number of Images')
plt.title('Number of Images per Class')

plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import precision_recall_fscore_support
import matplotlib.pyplot as plt
import pandas as pd

# Calculate precision, recall, and F1 scores for each class
precision, recall, f1, _ = precision_recall_fscore_support(test_df["Class Name"], pred_labels, average=None)

# Get the unique class labels
classes = np.unique(test_df["Class Name"])

# Sort precision, recall, and F1 scores in descending order
sorted_indices = np.argsort(f1)[::-1]
precision_sorted = precision[sorted_indices]
recall_sorted = recall[sorted_indices]
f1_sorted = f1[sorted_indices]
classes_sorted = classes[sorted_indices]
class_counts_sorted = class_counts[classes_sorted]

# Set the figure size
plt.figure(figsize=(12, 12))

# Plot the precision scores
plt.bar(range(len(classes)), precision_sorted, width=0.3, label='Precision', color='lightblue')
plt.xlabel('Class')
plt.ylabel('Score')
plt.title('Precision Scores by Class')
plt.xticks(range(len(classes)), classes_sorted, rotation='vertical')

# Add scores and class counts as text above each bar
for i, v in enumerate(precision_sorted):
    plt.text(i, v, f"{v:.2f}\n({class_counts_sorted[i]})", color='black', ha='center', va='bottom', fontsize=8)

plt.legend()
plt.tight_layout()
plt.show()

# Set the figure size
plt.figure(figsize=(12, 12))

# Plot the recall scores
plt.bar(range(len(classes)), recall_sorted, width=0.3, label='Recall', color='lightgreen')
plt.xlabel('Class')
plt.ylabel('Score')
plt.title('Recall Scores by Class')
plt.xticks(range(len(classes)), classes_sorted, rotation='vertical')

# Add scores and class counts as text above each bar
for i, v in enumerate(recall_sorted):
    plt.text(i, v, f"{v:.2f}\n({class_counts_sorted[i]})", color='black', ha='center', va='bottom', fontsize=8)

plt.legend()
plt.tight_layout()
plt.show()

# Set the figure size
plt.figure(figsize=(12, 12))

# Plot the F1 scores
plt.bar(range(len(classes)), f1_sorted, width=0.3, label='F1 Score', color='orange')
plt.xlabel('Class')
plt.ylabel('Score')
plt.title('F1 Scores by Class')
plt.xticks(range(len(classes)), classes_sorted, rotation='vertical')

# Add scores and class counts as text above each bar
for i, v in enumerate(f1_sorted):
    plt.text(i, v, f"{v:.2f}\n({class_counts_sorted[i]})", color='black', ha='center', va='bottom', fontsize=8)

plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
test_df.head()



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Calculate precision, recall, and F1 scores for each class
precision, recall, f1, _ = precision_recall_fscore_support(test_df["Class Name"], pred_labels, average=None)

# Calculate the mean F1 score
mean_f1 = np.mean(f1)

# Get the unique class labels and their counts
classes, class_counts = np.unique(test_df["Class Name"], return_counts=True)

# Sort precision, recall, F1 scores, and class counts in descending order
sorted_indices = np.argsort(f1)[::-1]
precision_sorted = precision[sorted_indices]
recall_sorted = recall[sorted_indices]
f1_sorted = f1[sorted_indices]
classes_sorted = classes[sorted_indices]
class_counts_sorted = class_counts[sorted_indices]

# Set the figure size and create subplots
fig, ax = plt.subplots(figsize=(12, 12))

# Set the width of the bars
bar_width = 0.8

# Plot the F1 scores
bars = ax.barh(np.arange(len(classes_sorted)), f1_sorted, height=bar_width, color='purple')

# Add F1 scores as text inside each bar
for i, (bar, score) in enumerate(zip(bars, f1_sorted)):
    ax.text(bar.get_width() + 0.02, bar.get_y() + bar.get_height() / 2, f"{score:.2f}",
            va='center', ha='left', fontsize=7)

# Set y-axis ticks and labels
ax.set_yticks(np.arange(len(classes_sorted)))
ax.set_yticklabels(classes_sorted, fontsize=7)

# Set x-axis label and limits
ax.set_xlabel('F1 Score', fontsize=10)
ax.set_xlim(0, 1)

# Add class counts as text near the class names
for i, count in enumerate(class_counts_sorted):
    ax.text(-0.07, i, f"  ({count})", va='center', ha='right', fontsize=8)

# Set plot title
ax.set_title('F1 Scores by Class', fontsize=12)

# Remove spines and ticks
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.tick_params(left=False, bottom=False)

# Add a grid to the plot
ax.grid(axis='x', color='lightgray', linestyle='--')

# Adjust layout
fig.tight_layout()

# Invert the y-axis
plt.gca().invert_yaxis()

# Show the plot
plt.show()


In [ ]:
class_iou_scores = []

# Calculate the mean IoU score for each class
for class_label in classes:
    # Get the indices of samples belonging to the current class
    class_indices = np.where(test_labels == class_label)[0]

    # Extract the predicted and ground truth bounding boxes for the current class
    class_pred_boxes = pred_boxes[class_indices]
    class_test_boxes = test_boxes[class_indices]

    # Calculate the IoU score for each sample
    iou_scores = []

    for i in range(len(class_indices)):
        iou_score = calculate_iou(class_test_boxes[i], class_pred_boxes[i])
        iou_scores.append(iou_score)

    # Calculate the mean IoU score for the current class
    mean_iou_score = np.mean(iou_scores)

    # Append the class label, mean IoU score, and number of images to the list of class IoU scores
    class_iou_scores.append((class_label, mean_iou_score, len(class_indices)))

# Sort the mean IoU scores from best to worst
class_iou_scores.sort(key=lambda x: x[1], reverse=True)

# Extract the class labels, mean IoU scores, and number of images into separate lists
sorted_classes = [x[0] for x in class_iou_scores]
sorted_scores = [x[1] for x in class_iou_scores]
num_images = [x[2] for x in class_iou_scores]

# Calculate the total mean IoU score
total_mean_iou = np.mean(sorted_scores)

# Plot the mean IoU scores for each class
fig, ax = plt.subplots(figsize=(12, 12))
bars = ax.barh(range(len(sorted_classes)), sorted_scores, color='purple')
ax.set_xlabel('Mean IoU Score')
ax.set_ylabel('Class')
ax.set_title('Mean IoU Scores by Class')
ax.set_yticks(range(len(sorted_classes)))
ax.set_yticklabels(sorted_classes)

for i, count in enumerate(class_counts_sorted):
    ax.text(-0.1, i, f"  ({count})", va='center', ha='right', fontsize=8)

# Add scores as text inside each bar
for bar, score in zip(bars, sorted_scores):
    ax.text(bar.get_width() + 0.005, bar.get_y() + bar.get_height() / 2, f"{score:.2f}",
            va='center', fontsize=7)

# Add total mean IoU score as text at the top
ax.text(0, len(sorted_classes), f"Total Mean IoU: {total_mean_iou:.2f}", ha='left', va='center', fontsize=12)

# Add a vertical line at x=0.5 for reference
ax.axvline(x=0.5, color='red', linestyle='--', linewidth=1)

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Calculate precision, recall, and F1 scores for each class
precision, recall, f1, _ = precision_recall_fscore_support(test_df["Class Name"], pred_labels, average=None)

# Calculate the mean F1 score
mean_f1 = np.mean(f1)

# Get the unique class labels and their counts
classes, class_counts = np.unique(test_df["Class Name"], return_counts=True)

# Sort precision, recall, F1 scores, and class counts in descending order based on F1 scores
sorted_indices = np.argsort(f1)[::-1]
precision_sorted = precision[sorted_indices]
recall_sorted = recall[sorted_indices]
f1_sorted = f1[sorted_indices]
classes_sorted = classes[sorted_indices]
class_counts_sorted = class_counts[sorted_indices]

# Calculate mean IoU score for each class
class_iou_scores = []

for class_label in classes_sorted:
    class_indices = np.where(test_labels == class_label)[0]
    class_pred_boxes = pred_boxes[class_indices]
    class_test_boxes = test_boxes[class_indices]

    iou_scores = []

    for i in range(len(class_indices)):
        iou_score = calculate_iou(class_test_boxes[i], class_pred_boxes[i])
        iou_scores.append(iou_score)

    mean_iou_score = np.mean(iou_scores)

    class_iou_scores.append(mean_iou_score)

# Sort the mean IoU scores from best to worst
class_iou_scores_sorted = np.array(class_iou_scores)[sorted_indices]

# Set the figure size and create subplots
fig, ax = plt.subplots(figsize=(12, 12))

# Set the width of the bars
bar_width = 0.4

# Plot the F1 scores
f1_bars = ax.barh(np.arange(len(classes_sorted)), f1_sorted, height=bar_width, color='purple', label='F1 Score')

# Plot the mean IoU scores
iou_bars = ax.barh(np.arange(len(classes_sorted)) + bar_width, class_iou_scores_sorted, height=bar_width, color='blue', label='Mean IoU')

# Add F1 scores as text inside each bar
for i, (bar, score) in enumerate(zip(f1_bars, f1_sorted)):
    ax.text(bar.get_width() + 0.02, bar.get_y() + bar.get_height() / 2, f"{score:.2f}",
            va='center', ha='left', fontsize=7)

# Add mean IoU scores as text inside each bar
for i, (bar, score) in enumerate(zip(iou_bars, class_iou_scores_sorted)):
    ax.text(bar.get_width() + 0.02, bar.get_y() + bar.get_height() / 2, f"{score:.2f}",
            va='center', ha='left', fontsize=7, color='white')

# Set y-axis ticks and labels
ax.set_yticks(np.arange(len(classes_sorted)) + bar_width / 2)
ax.set_yticklabels(classes_sorted, fontsize=7)

# Set x-axis label and limits
ax.set_xlabel('Scores', fontsize=10)
ax.set_xlim(0, 1)

# Add class counts as text near the class names
for i, count in enumerate(class_counts_sorted):
    ax.text(-0.07, i + bar_width / 2, f"  ({count})", va='center', ha='right', fontsize=8)

# Set plot title
ax.set_title('F1 Scores and Mean IoU Scores by Class', fontsize=12)

# Remove spines and ticks
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.tick_params(left=False, bottom=False)

# Add a grid to the plot
ax.grid(axis='x', color='lightgray', linestyle='--')

# Adjust layout
fig.tight_layout()

# Invert the y-axis
plt.gca().invert_yaxis()

# Show the legend
ax.legend(loc='lower right')

# Show the plot
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Calculate precision, recall, and F1 scores for each class
precision, recall, f1, _ = precision_recall_fscore_support(test_df["Class Name"], pred_labels, average=None)

# Calculate the mean F1 score
mean_f1 = np.mean(f1)

# Get the unique class labels and their counts
classes, class_counts = np.unique(test_df["Class Name"], return_counts=True)

# Sort precision, recall, F1 scores, and class counts in descending order based on F1 scores
sorted_indices = np.argsort(f1)[::-1]
precision_sorted = precision[sorted_indices]
recall_sorted = recall[sorted_indices]
f1_sorted = f1[sorted_indices]
classes_sorted = classes[sorted_indices]
class_counts_sorted = class_counts[sorted_indices]

# Calculate mean IoU score for each class
class_iou_scores = []

for class_label in classes_sorted:
    class_indices = np.where(test_labels == class_label)[0]
    class_pred_boxes = pred_boxes[class_indices]
    class_test_boxes = test_boxes[class_indices]

    iou_scores = []

    for i in range(len(class_indices)):
        iou_score = calculate_iou(class_test_boxes[i], class_pred_boxes[i])
        iou_scores.append(iou_score)

    mean_iou_score = np.mean(iou_scores)

    class_iou_scores.append(mean_iou_score)

# Sort the mean IoU scores from best to worst
class_iou_scores_sorted = np.array(class_iou_scores)[sorted_indices]

# Set the figure size and create subplots
fig, ax = plt.subplots(figsize=(12, 12))

# Set the width of the bars
bar_width = 0.4

# Plot the F1 scores
f1_bars = ax.barh(np.arange(len(classes_sorted)), f1_sorted, height=bar_width, color='green', label='F1 Score')

# Plot the mean IoU scores
iou_bars = ax.barh(np.arange(len(classes_sorted)) + bar_width, class_iou_scores_sorted, height=bar_width, color='blue', label='Mean IoU')

# Add F1 scores as text inside each bar
for i, (bar, score) in enumerate(zip(f1_bars, f1_sorted)):
    ax.text(bar.get_width() + 0.02, bar.get_y() + bar.get_height() / 2, f"{score:.2f}",
            va='center', ha='left', fontsize=7, fontweight='bold', color='red')

# Add mean IoU scores as text inside each bar
for i, (bar, score) in enumerate(zip(iou_bars, class_iou_scores_sorted)):
    ax.text(bar.get_width() + 0.02, bar.get_y() + bar.get_height() / 2, f"{score:.2f}",
            va='center', ha='left', fontsize=7, fontweight='bold', color='red')

# Set y-axis ticks and labels
ax.set_yticks(np.arange(len(classes_sorted)) + bar_width / 2)
ax.set_yticklabels(classes_sorted, fontsize=7, fontweight='bold', color='black')

# Set x-axis label and limits
ax.set_xlabel('Scores', fontsize=7, color='red')
ax.set_xlim(0, 1)

# Add class counts as text near the class names
for i, count in enumerate(class_counts_sorted):
    ax.text(-0.1, i + bar_width / 2, f"  ({count})", va='center', ha='right', fontsize=7, fontweight='bold', color='red')

# Set plot title
ax.set_title('F1 Scores and Mean IoU Scores by Class', fontsize=14, fontweight='bold', color='red')

# Remove spines and ticks
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.tick_params(left=False, bottom=False)

# Add a grid to the plot
ax.grid(axis='x', color='lightgray', linestyle='--')

# Show the legend
ax.legend(loc='upper right', fontsize=7)

# Invert the y-axis
plt.gca().invert_yaxis()

# Show the plot
plt.show()


In [ ]:
## WE ARE GOING TO SEPERATE CARSIDE DATASET FROM TRAINING ARRAYS:
## (WE ARE GOING TO ACCOMPLISH OUR EXPERIMENTS WITH 100 DATASET CLASSES)


## * RESNET50 --> MSE --> TRAINABLE = FALSE (TOP ONLY)
## * RESNET50 --> MSE --> TRAINABLE = TRUE (ONLY FULL)
## * RESNET50 --> MSE --> TRAINABLE = FALSE --> THEN --> TRAINABLE = TRUE (TRAIN SAME MODEL WITH SMALLER LEARNING RATE)


## SAME FOR IOU

## THEN SHOE THE RESULTS OF CARSIDE


## CREATE TABLE TO SHOW OUTPUTS IN BETTER WAY

In [ ]:
## LET'S IMPORT THE DATAFRAMES FOR VALIDATION, TEST AND TRAIN

train_path = '/content/drive/My Drive/train_df.csv'
train_df = pd.read_csv(train_path)

val_path = '/content/drive/My Drive/val_df.csv'
val_df = pd.read_csv(val_path)

test_path = '/content/drive/My Drive/test_df.csv'
test_df = pd.read_csv(test_path)

In [ ]:
print(len(train_df))
print(len(test_df))
print(len(val_df))


In [ ]:
import pandas as pd

# Filter rows with 'carside' class and create a new DataFrame
carside_df = train_df[train_df['Class Name'] == 'carside'].copy()

# Remove rows with 'carside' class from the original DataFrame
train_df = train_df[train_df['Class Name'] != 'carside']

# Reset the index of the new DataFrame
carside_df.reset_index(drop=True, inplace=True)

# Reset the index of the original DataFrame
train_df.reset_index(drop=True, inplace=True)

In [ ]:
print(len(carside_df))
print(len(train_df))

In [ ]:
import pandas as pd

# Filter rows with 'carside' class and create a new DataFrame
val_carside_df = val_df[val_df['Class Name'] == 'carside'].copy()

# Remove rows with 'carside' class from the original DataFrame
val_df = val_df[val_df['Class Name'] != 'carside']

# Reset the index of the new DataFrame
val_carside_df.reset_index(drop=True, inplace=True)

# Reset the index of the original DataFrame
val_df.reset_index(drop=True, inplace=True)

In [ ]:
print(len(val_carside_df))
print(len(val_df))

In [ ]:
import pandas as pd

# Filter rows with 'carside' class and create a new DataFrame
test_carside_df = test_df[test_df['Class Name'] == 'carside'].copy()

# Remove rows with 'carside' class from the original DataFrame
test_df = test_df[test_df['Class Name'] != 'carside']

# Reset the index of the new DataFrame
test_carside_df.reset_index(drop=True, inplace=True)

# Reset the index of the original DataFrame
test_df.reset_index(drop=True, inplace=True)

In [ ]:
print(len(test_carside_df))
print(len(test_df))

In [ ]:
import os

# Set the parent directory path
parent_dir = "/content/drive/MyDrive/"

# Set the directory name
directory_name = "project_mark1"

# Combine the parent directory path and directory name
directory_path = os.path.join(parent_dir, directory_name)

# Create the directory if it doesn't exist
os.makedirs(directory_path, exist_ok=True)

# Print the path of the created directory
print("Directory created:", directory_path)

In [ ]:
import os

# Set the parent directory path
parent_dir = "/content/drive/MyDrive/project_mark1"

# Save train_df to CSV
train_df.to_csv(os.path.join(parent_dir, "train_df.csv"), index=False)

# Save val_df to CSV
val_df.to_csv(os.path.join(parent_dir, "val_df.csv"), index=False)

# Save test_df to CSV
test_df.to_csv(os.path.join(parent_dir, "test_df.csv"), index=False)

# Save test_carside_df to CSV
test_carside_df.to_csv(os.path.join(parent_dir, "test_carside_df.csv"), index=False)

# Save val_carside_df to CSV
val_carside_df.to_csv(os.path.join(parent_dir, "val_carside_df.csv"), index=False)

# Save carside_df to CSV
carside_df.to_csv(os.path.join(parent_dir, "carside_df.csv"), index=False)

print("Files saved in directory:", parent_dir)

In [ ]:
#########################################
####### controls for 'carside' ##########

test_carside_df.head()

In [ ]:
val_carside_df.head()

In [ ]:
carside_df.head()

In [ ]:
# Count the occurrences of each class name in the train_df DataFrame
class_counts = train_df['Class Name'].value_counts()

# Check if the class name 'carside' exists in the DataFrame
if 'carside' in class_counts:
    # Get the number of classes with the class name 'carside'
    num_classes_carside = class_counts['carside']
    print(f"The number of classes with the class name 'carside': {num_classes_carside}")
else:
    print("No classes with the class name 'carside' found.")

In [ ]:
# Count the occurrences of each class name in the train_df DataFrame
class_counts = val_df['Class Name'].value_counts()

# Check if the class name 'carside' exists in the DataFrame
if 'carside' in class_counts:
    # Get the number of classes with the class name 'carside'
    num_classes_carside = class_counts['carside']
    print(f"The number of classes with the class name 'carside': {num_classes_carside}")
else:
    print("No classes with the class name 'carside' found.")

In [ ]:
# Count the occurrences of each class name in the train_df DataFrame
class_counts = test_df['Class Name'].value_counts()

# Check if the class name 'carside' exists in the DataFrame
if 'carside' in class_counts:
    # Get the number of classes with the class name 'carside'
    num_classes_carside = class_counts['carside']
    print(f"The number of classes with the class name 'carside': {num_classes_carside}")
else:
    print("No classes with the class name 'carside' found.")

In [ ]:
# Count the occurrences of each class name in the train_df DataFrame
class_counts = carside_df['Class Name'].value_counts()

# Check if the class name 'carside' exists in the DataFrame
if 'carside' in class_counts:
    # Get the number of classes with the class name 'carside'
    num_classes_carside = class_counts['carside']
    print(f"The number of classes with the class name 'carside': {num_classes_carside}")
else:
    print("No classes with the class name 'carside' found.")

In [ ]:
# Count the occurrences of each class name in the train_df DataFrame
class_counts = val_carside_df['Class Name'].value_counts()

# Check if the class name 'carside' exists in the DataFrame
if 'carside' in class_counts:
    # Get the number of classes with the class name 'carside'
    num_classes_carside = class_counts['carside']
    print(f"The number of classes with the class name 'carside': {num_classes_carside}")
else:
    print("No classes with the class name 'carside' found.")

In [ ]:
# Count the occurrences of each class name in the train_df DataFrame
class_counts = test_carside_df['Class Name'].value_counts()

# Check if the class name 'carside' exists in the DataFrame
if 'carside' in class_counts:
    # Get the number of classes with the class name 'carside'
    num_classes_carside = class_counts['carside']
    print(f"The number of classes with the class name 'carside': {num_classes_carside}")
else:
    print("No classes with the class name 'carside' found.")

In [ ]:
############################################################
######## okey we have now train - test - val df's ##########
############################################################
################# we also have carside dataset ###########
############################################################


## **100 classes resnet50 - mse - trainable = Ture**

In [ ]:
import tensorflow as tf

def iou_loss(y_true, y_pred):
    # Extract the normalized coordinates of the predicted and ground truth boxes
    pred_x1, pred_y1, pred_x2, pred_y2 = tf.unstack(y_pred, axis=1)
    true_x1, true_y1, true_x2, true_y2 = tf.unstack(y_true, axis=1)

    # Convert normalized coordinates to absolute coordinates
    pred_x1 = pred_x1 * 512
    pred_y1 = pred_y1 * 512
    pred_x2 = pred_x2 * 512
    pred_y2 = pred_y2 * 512
    true_x1 = true_x1 * 512
    true_y1 = true_y1 * 512
    true_x2 = true_x2 * 512
    true_y2 = true_y2 * 512

    # Calculate the coordinates of the intersection rectangle
    x1 = tf.maximum(pred_x1, true_x1)
    y1 = tf.maximum(pred_y1, true_y1)
    x2 = tf.minimum(pred_x2, true_x2)
    y2 = tf.minimum(pred_y2, true_y2)

    # Calculate the area of intersection
    intersection = tf.maximum(0.0, x2 - x1) * tf.maximum(0.0, y2 - y1)

    # Calculate the area of predicted and ground truth boxes
    pred_area = (pred_x2 - pred_x1) * (pred_y2 - pred_y1)
    true_area = (true_x2 - true_x1) * (true_y2 - true_y1)

    # Calculate the union area
    union = pred_area + true_area - intersection

    # Calculate the IoU
    iou = intersection / (union + tf.keras.backend.epsilon())

    # Calculate the IoU loss
    iou_loss = 1.0 - iou

    return iou_loss

In [ ]:
from keras.utils import get_custom_objects


losses = {
    "class_label": "categorical_crossentropy",
    "bounding_box": "iou_loss",
}

get_custom_objects().update({'iou_loss': iou_loss})


In [ ]:
####################################################
############## MOST IMPORTANT PART #################
####################################################
def resize_image(image, target_size):
    return cv2.resize(image, target_size)

def plot_images(images, boxes, labels, dataset_name):
    fig, axes = plt.subplots(1, len(images), figsize=(12, 4))
    for i, ax in enumerate(axes):
        ax.imshow(images[i])
        xmin, ymin, xmax, ymax = boxes[i]
        ax.add_patch(plt.Rectangle((xmin*224, ymin*224), xmax*224 - xmin*224, ymax*224 - ymin*224, fill=False, color='red', linewidth=2))
        ax.set_title(labels[i])
        ax.axis('off')
    plt.suptitle(f'{dataset_name} Images with Bounding Boxes')
    plt.show()

# Append data from val_df into arrays
val_images = []
val_boxes = []
val_labels = []

for index, row in val_df.iterrows():
    image_path = row['Image Path']
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = resize_image(image, (224, 224))
    val_images.append(image)

    xmin, ymin, xmax, ymax = map(float, [row['Xmin'], row['Ymin'], row['Xmax'], row['Ymax']])
    val_boxes.append([xmin, ymin, xmax, ymax])

    label = row['Class Name']
    val_labels.append(label)

# Convert the lists to NumPy arrays
val_images = np.array(val_images, dtype='float32')
val_boxes = np.array(val_boxes, dtype='float32')
val_labels = np.array(val_labels)

# Normalize the images
val_images = val_images / 255.0



# Append data from train_df into arrays
train_images = []
train_boxes = []
train_labels = []

for index, row in train_df.iterrows():
    image_path = row['Image Path']
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = resize_image(image, (224, 224))
    train_images.append(image)

    xmin, ymin, xmax, ymax = map(float, [row['Xmin'], row['Ymin'], row['Xmax'], row['Ymax']])
    train_boxes.append([xmin, ymin, xmax, ymax])

    label = row['Class Name']
    train_labels.append(label)

# Convert the lists to NumPy arrays
train_images = np.array(train_images, dtype='float32')
train_boxes = np.array(train_boxes, dtype='float32')
train_labels = np.array(train_labels)

# Normalize the images
train_images = train_images / 255.0



# Append data from test_df into arrays
test_images = []
test_boxes = []
test_labels = []

for index, row in test_df.iterrows():
    image_path = row['Image Path']
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = resize_image(image, (224, 224))
    test_images.append(image)

    xmin, ymin, xmax, ymax = map(float, [row['Xmin'], row['Ymin'], row['Xmax'], row['Ymax']])
    test_boxes.append([xmin, ymin, xmax, ymax])

    label = row['Class Name']
    test_labels.append(label)

# Convert the lists to NumPy arrays
test_images = np.array(test_images, dtype='float32')
test_boxes = np.array(test_boxes, dtype='float32')
test_labels = np.array(test_labels)

# Normalize the images
test_images = test_images / 255.0


In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Perform label encoding on the target labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
val_labels_encoded = label_encoder.transform(val_labels)

# Convert the encoded labels to one-hot encoding
train_labels_onehot = to_categorical(train_labels_encoded, num_classes=100) ## ==> update number of classes
val_labels_onehot = to_categorical(val_labels_encoded, num_classes=100) ## ==> update number of classes

# Model architecture
input_layer = Input(shape=(224, 224, 3))
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=input_layer)


flatten = Flatten()(base_model.output)


bboxHead = Dense(1024, activation="relu")(flatten)

bboxHead = Dense(512, activation="relu")(bboxHead)

bbox_output = Dense(4, activation='linear', name='bounding_box')(bboxHead)



softmaxHead = Dense(1024, activation="relu")(flatten)
softmaxHead = Dropout(0.5)(softmaxHead)

softmaxHead = Dense(512, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)

class_output = Dense(100, activation='softmax', name='class_output')(softmaxHead) ## ==> update number of classes

model_resnet_mse_1 = Model(inputs=input_layer, outputs=[class_output, bbox_output])



# Set the learning rate
learning_rate = 1e-4

# Create the optimizer with the desired learning rate
optimizer = Adam(learning_rate=learning_rate)


# Compile the model
model.compile(optimizer=optimizer, loss={'class_output': 'categorical_crossentropy', 'bounding_box': iou_loss})
model.summary()

In [ ]:
from keras.utils.vis_utils import plot_model

#Your Model code here use the Keras Sequential Architecture

plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
visualkeras.layered_view(model).show()


In [ ]:
visualkeras.layered_view(model, scale_xy=1, scale_z=1, max_z=100)


In [ ]:
visualkeras.layered_view(model, legend=True, draw_volume=False,spacing=30, scale_xy=1, scale_z=1, max_z=100)


In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)


In [ ]:
import graphviz


ann_viz(model , view=True, title= 'My Model')


In [ ]:
visualkeras.layered_view(model, legend=True)




In [ ]:
visualkeras.layered_view(model, scale_xy=1, scale_z=1, max_z=200)


In [ ]:
make_dot(model)


In [ ]:
tf.keras.utils.plot_model(
    model,
    to_file="model.png",
    show_shapes=False,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=96,
    layer_range=None,
    show_layer_activations=False,
    show_trainable=False,
)

In [ ]:
from keras.callbacks import EarlyStopping

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=7)


# Train the model with early stopping
model = model.fit(train_images, {'class_output': train_labels_onehot, 'bounding_box': train_boxes},
                    validation_data=(val_images, {'class_output': val_labels_onehot, 'bounding_box': val_boxes}),
                    epochs=300, batch_size=128, callbacks=[callback])

# Save the trained model
model.save('/content/drive/MyDrive/model__model.h5')


In [ ]:
import pickle

# Save the label encoder
with open('/content/drive/MyDrive/project_mark1/model_resnet_mse_1_label_encoder.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)

In [ ]:
# Plot the losses
plt.figure(figsize=(24, 12))
plt.style.use('seaborn')

# Classification Loss
plt.subplot(1, 2, 1)
plt.plot(model_resnet_mse_1_history.history['class_output_loss'], label='Train Classification Loss', color='blue')
plt.plot(model_resnet_mse_1_history.history['val_class_output_loss'], label='Validation Classification Loss', color='orange')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Classification Loss', fontsize=12)
plt.legend()

# Bounding Box Loss
plt.subplot(1, 2, 2)
plt.plot(model_resnet_mse_1_history.history['bounding_box_loss'], label='Train Bounding Box Loss', color='blue')
plt.plot(model_resnet_mse_1_history.history['val_bounding_box_loss'], label='Validation Bounding Box Loss', color='orange')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Bounding Box Loss', fontsize=12)
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Save the history object to a file using pickle
with open('model_resnet_mse_1_history.pkl', 'wb') as file:
    pickle.dump(model_resnet_mse_1_history.history, file)

In [ ]:
# Save the entire model to a HDF5 file
model_resnet_mse_1.save('/content/drive/MyDrive/project_mark1/model_resnet_mse_1.h5')

In [ ]:
# Save the history object to a file using pickle
with open('/content/drive/MyDrive/project_mark1/model_resnet_mse_1_history.pkl', 'wb') as file:
    pickle.dump(model_resnet_mse_1_history.history, file)


In [ ]:
### --> instead of iou names I have the mse names for results of iou

## **100 classes resnet50 - mse - trainable = false**

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Perform label encoding on the target labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
val_labels_encoded = label_encoder.transform(val_labels)

# Convert the encoded labels to one-hot encoding
train_labels_onehot = to_categorical(train_labels_encoded, num_classes=100) ## ==> update number of classes
val_labels_onehot = to_categorical(val_labels_encoded, num_classes=100) ## ==> update number of classes

# Model architecture
input_layer = Input(shape=(224, 224, 3))
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=input_layer)


base_model.trainable = False


flatten = Flatten()(base_model.output)


bboxHead = Dense(1024, activation="relu")(flatten)

bboxHead = Dense(512, activation="relu")(bboxHead)

bboxHead = Dense(256, activation="relu")(bboxHead)

bboxHead = Dense(128, activation="relu")(bboxHead)

bboxHead = Dense(64, activation="relu")(bboxHead)


bboxHead = Dense(32, activation="relu")(bboxHead)



bbox_output = Dense(4, activation='linear', name='bounding_box')(bboxHead)




softmaxHead = Dense(1024, activation="relu")(flatten)
softmaxHead = Dropout(0.5)(softmaxHead)

softmaxHead = Dense(512, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)

softmaxHead = Dense(256, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)

softmaxHead = Dense(128, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)

class_output = Dense(100, activation='softmax', name='class_output')(softmaxHead) ## ==> update number of classes



model_resnet_mse_2 = Model(inputs=input_layer, outputs=[class_output, bbox_output])

# Set the learning rate
learning_rate = 1e-4

# Create the optimizer with the desired learning rate
optimizer = Adam(learning_rate=learning_rate)


# Compile the model
model_resnet_mse_2.compile(optimizer= optimizer, loss={'class_output': 'categorical_crossentropy', 'bounding_box': iou_loss})

model_resnet_mse_2.summary()

In [ ]:
## at least we understand how we are going to train them:

In [ ]:
from keras.callbacks import EarlyStopping

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=7)


# Train the model with early stopping
model_resnet_mse_2_history = model_resnet_mse_2.fit(train_images, {'class_output': train_labels_onehot, 'bounding_box': train_boxes},
                    validation_data=(val_images, {'class_output': val_labels_onehot, 'bounding_box': val_boxes}),
                    epochs=300, batch_size=32, callbacks=[callback])

# Save the trained model
model_resnet_mse_2.save('/content/drive/MyDrive/project_mark1/model_resnet_mse_2_trained_model.h5')

In [ ]:
# Plot the losses
plt.figure(figsize=(12, 7))
plt.style.use('seaborn-whitegrid')

# Classification Loss
plt.subplot(1, 2, 1)
plt.plot(model_resnet_mse_2_history.history['class_output_loss'], label='Train Classification Loss', color='blue')
plt.plot(model_resnet_mse_2_history.history['val_class_output_loss'], label='Validation Classification Loss', color='orange')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Classification Loss', fontsize=12)
plt.legend()

# Bounding Box Loss
plt.subplot(1, 2, 2)
plt.plot(model_resnet_mse_2_history.history['bounding_box_loss'], label='Train Bounding Box Loss', color='blue')
plt.plot(model_resnet_mse_2_history.history['val_bounding_box_loss'], label='Validation Bounding Box Loss', color='orange')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Bounding Box Loss', fontsize=12)
plt.legend()

# Set plot limits and tick parameters
num_epochs = len(model_resnet_mse_2_history.history['class_output_loss'])
max_loss = max(max(model_resnet_mse_2_history.history['class_output_loss']), max(model_resnet_mse_2_history.history['bounding_box_loss']))
plt.xlim(0, num_epochs - 1)
plt.ylim(0, max_loss)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

# Add annotations for the first and last values
first_class_loss = model_resnet_mse_2_history.history['class_output_loss'][0]
last_class_loss = model_resnet_mse_2_history.history['class_output_loss'][-1]
first_bbox_loss = model_resnet_mse_2_history.history['bounding_box_loss'][0]
last_bbox_loss = model_resnet_mse_2_history.history['bounding_box_loss'][-1]
plt.annotate(f'First: {first_class_loss:.4f}', xy=(0, first_class_loss), xytext=(10, 5), textcoords='offset points', color='blue', fontsize=10)
plt.annotate(f'Last: {last_class_loss:.4f}', xy=(num_epochs - 1, last_class_loss), xytext=(-60, -15), textcoords='offset points', color='blue', fontsize=10)
plt.annotate(f'First: {first_bbox_loss:.4f}', xy=(0, first_bbox_loss), xytext=(10, 5), textcoords='offset points', color='blue', fontsize=10)
plt.annotate(f'Last: {last_bbox_loss:.4f}', xy=(num_epochs - 1, last_bbox_loss), xytext=(-60, -15), textcoords='offset points', color='blue', fontsize=10)

# Add a grid for better readability
plt.grid(color='lightgray')

# Adjust the layout to avoid overlapping annotations
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
import pickle

# Save the label encoder
with open('/content/drive/MyDrive/project_mark1/model_resnet_mse_2_label_encoder.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)

In [ ]:
# Plot the losses
plt.figure(figsize=(24, 12))
plt.style.use('seaborn')

# Classification Loss
plt.subplot(1, 2, 1)
plt.plot(model_resnet_mse_2_history.history['class_output_loss'], label='Train Classification Loss', color='blue')
plt.plot(model_resnet_mse_2_history.history['val_class_output_loss'], label='Validation Classification Loss', color='orange')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Classification Loss', fontsize=12)
plt.legend()

# Bounding Box Loss
plt.subplot(1, 2, 2)
plt.plot(model_resnet_mse_2_history.history['bounding_box_loss'], label='Train Bounding Box Loss', color='blue')
plt.plot(model_resnet_mse_2_history.history['val_bounding_box_loss'], label='Validation Bounding Box Loss', color='orange')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Bounding Box Loss', fontsize=12)
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Save the history object to a file using pickle
with open('model_resnet_mse_2_history.pkl', 'wb') as file:
    pickle.dump(model_resnet_mse_2_history.history, file)

In [ ]:
# Save the entire model to a HDF5 file
model_resnet_mse_2.save('/content/drive/MyDrive/project_mark1/model_resnet_mse_2.h5')

In [ ]:
# Save the history object to a file using pickle
with open('/content/drive/MyDrive/project_mark1/model_resnet_mse_2_history.pkl', 'wb') as file:
    pickle.dump(model_resnet_mse_2_history.history, file)

## **100 classes resnet50 - mse - trainable = false - then - trainable = true with lower learning rate**

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout
from keras.applications import ResNet50

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)            ### --> Canan Hocaya sor ???

# Perform label encoding on the target labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
val_labels_encoded = label_encoder.transform(val_labels)

# Convert the encoded labels to one-hot encoding
num_classes = 100  # Update with the actual number of classes
train_labels_onehot = to_categorical(train_labels_encoded, num_classes=num_classes)
val_labels_onehot = to_categorical(val_labels_encoded, num_classes=num_classes)

# Model architecture
input_layer = Input(shape=(224, 224, 3))
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=input_layer)

flatten = Flatten()(base_model.output)

# Top layers for bounding box regression
bboxHead = Dense(1024, activation="relu")(flatten)
# bboxHead = Dense(1024, activation="relu")(bboxHead)
# bboxHead = Dense(1024, activation="relu")(bboxHead)


# bboxHead = Dense(512, activation="relu")(bboxHead)
# bboxHead = Dense(512, activation="relu")(bboxHead)
bboxHead = Dense(512, activation="relu")(bboxHead)

# bboxHead = Dense(256, activation="relu")(bboxHead)
# bboxHead = Dense(256, activation="relu")(bboxHead)
bboxHead = Dense(256, activation="relu")(bboxHead)

# bboxHead = Dense(128, activation="relu")(bboxHead)
# bboxHead = Dense(128, activation="relu")(bboxHead)
bboxHead = Dense(128, activation="relu")(bboxHead)

# bboxHead = Dense(64, activation="relu")(bboxHead)
# bboxHead = Dense(64, activation="relu")(bboxHead)
bboxHead = Dense(64, activation="relu")(bboxHead)

# bboxHead = Dense(32, activation="relu")(bboxHead)
# bboxHead = Dense(32, activation="relu")(bboxHead)
bboxHead = Dense(32, activation="relu")(bboxHead)

# bboxHead = Dense(16, activation="relu")(bboxHead)
# bboxHead = Dense(16, activation="relu")(bboxHead)
# bboxHead = Dense(16, activation="relu")(bboxHead)

# bboxHead = Dense(8, activation="relu")(bboxHead)
# bboxHead = Dense(8, activation="relu")(bboxHead)
# bboxHead = Dense(8, activation="relu")(bboxHead)


bbox_output = Dense(4, activation='linear', name='bounding_box')(bboxHead)

# Top layers for classification
softmaxHead = Dense(1024, activation="relu")(flatten)
softmaxHead = Dropout(0.5)(softmaxHead)
softmaxHead = Dense(512, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)
softmaxHead = Dense(256, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)
softmaxHead = Dense(128, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)
class_output = Dense(num_classes, activation='softmax', name='class_output')(softmaxHead)

# Create the first model (train only the top layers)
model_top_layers = Model(inputs=input_layer, outputs=[class_output, bbox_output])

# Freeze the base model's layers
for layer in base_model.layers:
    layer.trainable = False

learning_rate = 1e-4

# Compile the model for training only the top layers
model_top_layers.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss={'class_output': 'categorical_crossentropy', 'bounding_box': iou_loss})

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=7)


# Train only the top layers
model_top_layers_history = model_top_layers.fit(train_images, {'class_output': train_labels_onehot, 'bounding_box': train_boxes},
                                                validation_data=(val_images, {'class_output': val_labels_onehot, 'bounding_box': val_boxes}),
                                                epochs=300, batch_size=32, callbacks=[callback])



# Unfreeze the base model's layers
for layer in base_model.layers:
    layer.trainable = True



# Recompile the model with a smaller learning rate for fine-tuning
learning_rate = 1e-5  # Update with the desired learning rate
model_top_layers.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss={'class_output': 'categorical_crossentropy', 'bounding_box': iou_loss})              #### --> Canan Hocaya sor ???

# Train the model with fine-tuning
model_fine_tuned_history = model_top_layers.fit(train_images, {'class_output': train_labels_onehot, 'bounding_box': train_boxes},
                                                validation_data=(val_images, {'class_output': val_labels_onehot, 'bounding_box': val_boxes}),
                                                epochs=300, batch_size=32, callbacks=[callback])


In [ ]:
# Save the label encoder
with open('/content/drive/MyDrive/project_mark1/model_top_layers_label_encoder.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)

# Save the entire model to a HDF5 file
model_top_layers.save('/content/drive/MyDrive/project_mark1/model_top_layers.h5')

# Save the history object to a file using pickle
with open('/content/drive/MyDrive/project_mark1/model_fine_tuned_history.pkl', 'wb') as file:
    pickle.dump(model_fine_tuned_history.history, file)

In [ ]:
# Plot the losses
plt.figure(figsize=(24, 12))
plt.style.use('seaborn')

# Classification Loss
plt.subplot(1, 2, 1)
plt.plot(model_fine_tuned_history.history['class_output_loss'], label='Train Classification Loss', color='blue')
plt.plot(model_fine_tuned_history.history['val_class_output_loss'], label='Validation Classification Loss', color='orange')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Classification Loss', fontsize=12)
plt.legend()

# Bounding Box Loss
plt.subplot(1, 2, 2)
plt.plot(model_fine_tuned_history.history['bounding_box_loss'], label='Train Bounding Box Loss', color='blue')
plt.plot(model_fine_tuned_history.history['val_bounding_box_loss'], label='Validation Bounding Box Loss', color='orange')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Bounding Box Loss', fontsize=12)
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Plot the losses
plt.figure(figsize=(12, 12))
plt.style.use('seaborn-whitegrid')

# Classification Loss
plt.subplot(1, 2, 1)
plt.plot(model_resnet_mse_1_history.history['class_output_loss'], label='Train Classification Loss', color='blue')
plt.plot(model_resnet_mse_1_history.history['val_class_output_loss'], label='Validation Classification Loss', color='orange')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Classification Loss', fontsize=12)
plt.legend()

# Bounding Box Loss
plt.subplot(1, 2, 2)
plt.plot(model_resnet_mse_1_history.history['bounding_box_loss'], label='Train Bounding Box Loss', color='blue')
plt.plot(model_resnet_mse_1_history.history['val_bounding_box_loss'], label='Validation Bounding Box Loss', color='orange')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Bounding Box Loss', fontsize=12)
plt.legend()

# Set plot limits and tick parameters
num_epochs = len(model_resnet_mse_1_history.history['class_output_loss'])
max_loss = max(max(model_resnet_mse_1_history.history['class_output_loss']), max(model_resnet_mse_1_history.history['bounding_box_loss']))
plt.xlim(0, num_epochs - 1)
plt.ylim(0, max_loss)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

# Add annotations for the first and last values
first_class_loss = model_resnet_mse_1_history.history['class_output_loss'][0]
last_class_loss = model_resnet_mse_1_history.history['class_output_loss'][-1]
first_bbox_loss = model_resnet_mse_1_history.history['bounding_box_loss'][0]
last_bbox_loss = model_resnet_mse_1_history.history['bounding_box_loss'][-1]
plt.annotate(f'First: {first_class_loss:.4f}', xy=(0, first_class_loss), xytext=(10, 5), textcoords='offset points', color='blue', fontsize=10)
plt.annotate(f'Last: {last_class_loss:.4f}', xy=(num_epochs - 1, last_class_loss), xytext=(-60, -15), textcoords='offset points', color='blue', fontsize=10)
plt.annotate(f'First: {first_bbox_loss:.4f}', xy=(0, first_bbox_loss), xytext=(10, 5), textcoords='offset points', color='blue', fontsize=10)
plt.annotate(f'Last: {last_bbox_loss:.4f}', xy=(num_epochs - 1, last_bbox_loss), xytext=(-60, -15), textcoords='offset points', color='blue', fontsize=10)

# Add a grid for better readability
plt.grid(color='lightgray')

# Adjust the layout to avoid overlapping annotations
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
###############################################
################ MSE - 1 ######################
###############################################

In [ ]:
import numpy as np
import tensorflow as tf

# Load the trained model
model_resnet_mse_1 = tf.keras.models.load_model('/content/drive/MyDrive/project_mark1/model_resnet_mse_1_trained_model.h5')


In [ ]:
test_labels
test_images
test_boxes


In [ ]:
file_path = '/content/caltech_files/caltech-101_new/classes.txt'

# Read the class names from the file
with open(file_path, 'r') as file:
    class_names = [line.strip() for line in file.readlines()]

# Print the class names
print(class_names)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Read the class names from the file
with open(file_path, 'r') as file:
    class_names = [line.strip() for line in file.readlines()]

# Create the Label Encoder
label_encoder = LabelEncoder()

# Fit the Label Encoder on the class names
label_encoder.fit(class_names)

# Print the encoded classes
encoded_classes = label_encoder.transform(class_names)
print(encoded_classes)

# Print the original class names and their corresponding encoded values
for class_name, encoded_value in zip(class_names, encoded_classes):
    print(f"{class_name}: {encoded_value}")

# Save the Label Encoder to a file
output_path = '/content/drive/MyDrive/project_mark1/label_encoder.pkl'
with open(output_path, 'wb') as file:
    pickle.dump(label_encoder, file)

In [ ]:
# Perform predictions on the test data
preds = model_resnet_mse_1.predict(test_images)
pred_labels = label_encoder.inverse_transform(np.argmax(preds[0], axis=1))
pred_boxes = preds[1]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support

# Calculate precision, recall, and F1 scores for each class
precision, recall, f1, _ = precision_recall_fscore_support(test_df["Class Name"], pred_labels, average=None)


In [ ]:
import matplotlib.patches as patches
from sklearn.metrics import jaccard_score

# Define a function to calculate the IoU score
def calculate_iou(box1, box2):
    # Extract coordinates from the bounding boxes
    xmin1, ymin1, xmax1, ymax1 = box1
    xmin2, ymin2, xmax2, ymax2 = box2

    # Calculate the coordinates of the intersection rectangle
    xmin_inter = max(xmin1, xmin2)
    ymin_inter = max(ymin1, ymin2)
    xmax_inter = min(xmax1, xmax2)
    ymax_inter = min(ymax1, ymax2)

    # Calculate the area of intersection rectangle
    inter_area = max(0, xmax_inter - xmin_inter + 1) * max(0, ymax_inter - ymin_inter + 1)

    # Calculate the areas of the bounding boxes
    box1_area = (xmax1 - xmin1 + 1) * (ymax1 - ymin1 + 1)
    box2_area = (xmax2 - xmin2 + 1) * (ymax2 - ymin2 + 1)

    # Calculate the IoU score
    iou = inter_area / float(box1_area + box2_area - inter_area)
    return iou


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Calculate precision, recall, and F1 scores for each class
precision, recall, f1, _ = precision_recall_fscore_support(test_df["Class Name"], pred_labels, average=None)

# Calculate the mean F1 score
mean_f1 = np.mean(f1)

# Get the unique class labels and their counts
classes, class_counts = np.unique(test_df["Class Name"], return_counts=True)

# Sort precision, recall, F1 scores, and class counts in descending order based on F1 scores
sorted_indices = np.argsort(f1)[::-1]
precision_sorted = precision[sorted_indices]
recall_sorted = recall[sorted_indices]
f1_sorted = f1[sorted_indices]
classes_sorted = classes[sorted_indices]
class_counts_sorted = class_counts[sorted_indices]

# Calculate mean IoU score for each class
class_iou_scores = []

for class_label in classes_sorted:
    class_indices = np.where(test_labels == class_label)[0]
    class_pred_boxes = pred_boxes[class_indices]
    class_test_boxes = test_boxes[class_indices]

    iou_scores = []

    for i in range(len(class_indices)):
        iou_score = calculate_iou(class_test_boxes[i], class_pred_boxes[i])
        iou_scores.append(iou_score)

    mean_iou_score = np.mean(iou_scores)

    class_iou_scores.append(mean_iou_score)

# Sort the mean IoU scores from best to worst
class_iou_scores_sorted = np.array(class_iou_scores)[sorted_indices]

# Set the figure size and create subplots
fig, ax = plt.subplots(figsize=(12, 12))

# Set the width of the bars
bar_width = 0.4

# Plot the F1 scores
f1_bars = ax.barh(np.arange(len(classes_sorted)), f1_sorted, height=bar_width, color='purple', label='F1 Score')

# Plot the mean IoU scores
iou_bars = ax.barh(np.arange(len(classes_sorted)) + bar_width, class_iou_scores_sorted, height=bar_width, color='blue', label='Mean IoU')

# Add F1 scores as text inside each bar
for i, (bar, score) in enumerate(zip(f1_bars, f1_sorted)):
    ax.text(bar.get_width() + 0.02, bar.get_y() + bar.get_height() / 2, f"{score:.2f}",
            va='center', ha='left', fontsize=7, fontweight='bold', color='red')

# Add mean IoU scores as text inside each bar
for i, (bar, score) in enumerate(zip(iou_bars, class_iou_scores_sorted)):
    ax.text(bar.get_width() + 0.02, bar.get_y() + bar.get_height() / 2, f"{score:.2f}",
            va='center', ha='left', fontsize=7, fontweight='bold', color='red')

# Set y-axis ticks and labels
ax.set_yticks(np.arange(len(classes_sorted)) + bar_width / 2)
ax.set_yticklabels(classes_sorted, fontsize=7, fontweight='bold', color='black')

# Set x-axis label and limits
ax.set_xlabel('Scores', fontsize=7, color='red')
ax.set_xlim(0, 1)

# Add class counts as text near the class names
for i, count in enumerate(class_counts_sorted):
    ax.text(-0.1, i + bar_width / 2, f"  ({count})", va='center', ha='right', fontsize=7, fontweight='bold', color='red')

# Set plot title
ax.set_title('F1 Scores and Mean IoU Scores by Class', fontsize=14, fontweight='bold', color='red')

# Remove spines and ticks
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.tick_params(left=False, bottom=False)

# Add a grid to the plot
ax.grid(axis='x', color='lightgray', linestyle='--')

# Show the legend
ax.legend(loc='upper right', fontsize=7)

# Invert the y-axis
plt.gca().invert_yaxis()

# Show the plot
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support

# Calculate precision, recall, and F1 scores for each class
precision, recall, f1, _ = precision_recall_fscore_support(test_df["Class Name"], pred_labels, average=None)

# Calculate mean F1 score
mean_f1 = np.mean(f1)

# Get the unique class labels and their counts
classes, class_counts = np.unique(test_df["Class Name"], return_counts=True)

# Sort precision, recall, F1 scores, and class counts in descending order based on F1 scores
sorted_indices = np.argsort(f1)[::-1]
precision_sorted = precision[sorted_indices]
recall_sorted = recall[sorted_indices]
f1_sorted = f1[sorted_indices]
classes_sorted = classes[sorted_indices]
class_counts_sorted = class_counts[sorted_indices]

# Calculate mean IoU score for each class
class_iou_scores = []

for class_label in classes_sorted:
    class_indices = np.where(test_labels == class_label)[0]
    class_pred_boxes = pred_boxes[class_indices]
    class_test_boxes = test_boxes[class_indices]

    iou_scores = []

    for i in range(len(class_indices)):
        iou_score = calculate_iou(class_test_boxes[i], class_pred_boxes[i])
        iou_scores.append(iou_score)

    mean_iou_score = np.mean(iou_scores)

    class_iou_scores.append(mean_iou_score)

# Sort the mean IoU scores from best to worst
class_iou_scores_sorted = np.array(class_iou_scores)[sorted_indices]

# Create a DataFrame to store the results
results_df_1 = pd.DataFrame({
    'Class': classes_sorted,
    'F1 Score': f1_sorted,
    'Mean IoU': class_iou_scores_sorted,
    'Class Count': class_counts_sorted
})

# Save the results to a CSV file
results_df_1.to_csv('results_1.csv', index=False)

# Display the DataFrame
print(results_df_1)

In [ ]:
##########################################################
################## MSE -2 ################################

In [ ]:
# Perform predictions on the test data
preds = model_resnet_mse_2.predict(test_images)
pred_labels = label_encoder.inverse_transform(np.argmax(preds[0], axis=1))
pred_boxes = preds[1]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support

# Calculate precision, recall, and F1 scores for each class
precision, recall, f1, _ = precision_recall_fscore_support(test_df["Class Name"], pred_labels, average=None)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Calculate precision, recall, and F1 scores for each class
precision, recall, f1, _ = precision_recall_fscore_support(test_df["Class Name"], pred_labels, average=None)

# Calculate the mean F1 score
mean_f1 = np.mean(f1)

# Get the unique class labels and their counts
classes, class_counts = np.unique(test_df["Class Name"], return_counts=True)

# Sort precision, recall, F1 scores, and class counts in descending order based on F1 scores
sorted_indices = np.argsort(f1)[::-1]
precision_sorted = precision[sorted_indices]
recall_sorted = recall[sorted_indices]
f1_sorted = f1[sorted_indices]
classes_sorted = classes[sorted_indices]
class_counts_sorted = class_counts[sorted_indices]

# Calculate mean IoU score for each class
class_iou_scores = []

for class_label in classes_sorted:
    class_indices = np.where(test_labels == class_label)[0]
    class_pred_boxes = pred_boxes[class_indices]
    class_test_boxes = test_boxes[class_indices]

    iou_scores = []

    for i in range(len(class_indices)):
        iou_score = calculate_iou(class_test_boxes[i], class_pred_boxes[i])
        iou_scores.append(iou_score)

    mean_iou_score = np.mean(iou_scores)

    class_iou_scores.append(mean_iou_score)

# Sort the mean IoU scores from best to worst
class_iou_scores_sorted = np.array(class_iou_scores)[sorted_indices]

# Set the figure size and create subplots
fig, ax = plt.subplots(figsize=(12, 12))

# Set the width of the bars
bar_width = 0.4

# Plot the F1 scores
f1_bars = ax.barh(np.arange(len(classes_sorted)), f1_sorted, height=bar_width, color='purple', label='F1 Score')

# Plot the mean IoU scores
iou_bars = ax.barh(np.arange(len(classes_sorted)) + bar_width, class_iou_scores_sorted, height=bar_width, color='blue', label='Mean IoU')

# Add F1 scores as text inside each bar
for i, (bar, score) in enumerate(zip(f1_bars, f1_sorted)):
    ax.text(bar.get_width() + 0.02, bar.get_y() + bar.get_height() / 2, f"{score:.2f}",
            va='center', ha='left', fontsize=7, fontweight='bold', color='red')

# Add mean IoU scores as text inside each bar
for i, (bar, score) in enumerate(zip(iou_bars, class_iou_scores_sorted)):
    ax.text(bar.get_width() + 0.02, bar.get_y() + bar.get_height() / 2, f"{score:.2f}",
            va='center', ha='left', fontsize=7, fontweight='bold', color='red')

# Set y-axis ticks and labels
ax.set_yticks(np.arange(len(classes_sorted)) + bar_width / 2)
ax.set_yticklabels(classes_sorted, fontsize=7, fontweight='bold', color='black')

# Set x-axis label and limits
ax.set_xlabel('Scores', fontsize=7, color='red')
ax.set_xlim(0, 1)

# Add class counts as text near the class names
for i, count in enumerate(class_counts_sorted):
    ax.text(-0.1, i + bar_width / 2, f"  ({count})", va='center', ha='right', fontsize=7, fontweight='bold', color='red')

# Set plot title
ax.set_title('F1 Scores and Mean IoU Scores by Class', fontsize=14, fontweight='bold', color='red')

# Remove spines and ticks
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.tick_params(left=False, bottom=False)

# Add a grid to the plot
ax.grid(axis='x', color='lightgray', linestyle='--')

# Show the legend
ax.legend(loc='upper right', fontsize=7)

# Invert the y-axis
plt.gca().invert_yaxis()

# Show the plot
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support

# Calculate precision, recall, and F1 scores for each class
precision, recall, f1, _ = precision_recall_fscore_support(test_df["Class Name"], pred_labels, average=None)

# Calculate mean F1 score
mean_f1 = np.mean(f1)

# Get the unique class labels and their counts
classes, class_counts = np.unique(test_df["Class Name"], return_counts=True)

# Sort precision, recall, F1 scores, and class counts in descending order based on F1 scores
sorted_indices = np.argsort(f1)[::-1]
precision_sorted = precision[sorted_indices]
recall_sorted = recall[sorted_indices]
f1_sorted = f1[sorted_indices]
classes_sorted = classes[sorted_indices]
class_counts_sorted = class_counts[sorted_indices]

# Calculate mean IoU score for each class
class_iou_scores = []

for class_label in classes_sorted:
    class_indices = np.where(test_labels == class_label)[0]
    class_pred_boxes = pred_boxes[class_indices]
    class_test_boxes = test_boxes[class_indices]

    iou_scores = []

    for i in range(len(class_indices)):
        iou_score = calculate_iou(class_test_boxes[i], class_pred_boxes[i])
        iou_scores.append(iou_score)

    mean_iou_score = np.mean(iou_scores)

    class_iou_scores.append(mean_iou_score)

# Sort the mean IoU scores from best to worst
class_iou_scores_sorted = np.array(class_iou_scores)[sorted_indices]

# Create a DataFrame to store the results
results_df_2 = pd.DataFrame({
    'Class': classes_sorted,
    'F1 Score': f1_sorted,
    'Mean IoU': class_iou_scores_sorted,
    'Class Count': class_counts_sorted
})

# Save the results to a CSV file
results_df_2.to_csv('results_2.csv', index=False)

# Display the DataFrame
print(results_df_2)

In [ ]:
################################################################
###################### MSE -3 ##################################

In [ ]:
# Perform predictions on the test data
preds = model_top_layers.predict(test_images)
pred_labels = label_encoder.inverse_transform(np.argmax(preds[0], axis=1))
pred_boxes = preds[1]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support

# Calculate precision, recall, and F1 scores for each class
precision, recall, f1, _ = precision_recall_fscore_support(test_df["Class Name"], pred_labels, average=None)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Calculate precision, recall, and F1 scores for each class
precision, recall, f1, _ = precision_recall_fscore_support(test_df["Class Name"], pred_labels, average=None)

# Calculate the mean F1 score
mean_f1 = np.mean(f1)

# Get the unique class labels and their counts
classes, class_counts = np.unique(test_df["Class Name"], return_counts=True)

# Sort precision, recall, F1 scores, and class counts in descending order based on F1 scores
sorted_indices = np.argsort(f1)[::-1]
precision_sorted = precision[sorted_indices]
recall_sorted = recall[sorted_indices]
f1_sorted = f1[sorted_indices]
classes_sorted = classes[sorted_indices]
class_counts_sorted = class_counts[sorted_indices]

# Calculate mean IoU score for each class
class_iou_scores = []

for class_label in classes_sorted:
    class_indices = np.where(test_labels == class_label)[0]
    class_pred_boxes = pred_boxes[class_indices]
    class_test_boxes = test_boxes[class_indices]

    iou_scores = []

    for i in range(len(class_indices)):
        iou_score = calculate_iou(class_test_boxes[i], class_pred_boxes[i])
        iou_scores.append(iou_score)

    mean_iou_score = np.mean(iou_scores)

    class_iou_scores.append(mean_iou_score)

# Sort the mean IoU scores from best to worst
class_iou_scores_sorted = np.array(class_iou_scores)[sorted_indices]

# Set the figure size and create subplots
fig, ax = plt.subplots(figsize=(12, 12))

# Set the width of the bars
bar_width = 0.4

# Plot the F1 scores
f1_bars = ax.barh(np.arange(len(classes_sorted)), f1_sorted, height=bar_width, color='purple', label='F1 Score')

# Plot the mean IoU scores
iou_bars = ax.barh(np.arange(len(classes_sorted)) + bar_width, class_iou_scores_sorted, height=bar_width, color='blue', label='Mean IoU')

# Add F1 scores as text inside each bar
for i, (bar, score) in enumerate(zip(f1_bars, f1_sorted)):
    ax.text(bar.get_width() + 0.02, bar.get_y() + bar.get_height() / 2, f"{score:.2f}",
            va='center', ha='left', fontsize=7, fontweight='bold', color='red')

# Add mean IoU scores as text inside each bar
for i, (bar, score) in enumerate(zip(iou_bars, class_iou_scores_sorted)):
    ax.text(bar.get_width() + 0.02, bar.get_y() + bar.get_height() / 2, f"{score:.2f}",
            va='center', ha='left', fontsize=7, fontweight='bold', color='red')

# Set y-axis ticks and labels
ax.set_yticks(np.arange(len(classes_sorted)) + bar_width / 2)
ax.set_yticklabels(classes_sorted, fontsize=7, fontweight='bold', color='black')

# Set x-axis label and limits
ax.set_xlabel('Scores', fontsize=7, color='red')
ax.set_xlim(0, 1)

# Add class counts as text near the class names
for i, count in enumerate(class_counts_sorted):
    ax.text(-0.1, i + bar_width / 2, f"  ({count})", va='center', ha='right', fontsize=7, fontweight='bold', color='red')

# Set plot title
ax.set_title('F1 Scores and Mean IoU Scores by Class', fontsize=14, fontweight='bold', color='red')

# Remove spines and ticks
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.tick_params(left=False, bottom=False)

# Add a grid to the plot
ax.grid(axis='x', color='lightgray', linestyle='--')

# Show the legend
ax.legend(loc='upper right', fontsize=7)

# Invert the y-axis
plt.gca().invert_yaxis()

# Show the plot
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support

# Calculate precision, recall, and F1 scores for each class
precision, recall, f1, _ = precision_recall_fscore_support(test_df["Class Name"], pred_labels, average=None)

# Calculate mean F1 score
mean_f1 = np.mean(f1)

# Get the unique class labels and their counts
classes, class_counts = np.unique(test_df["Class Name"], return_counts=True)

# Sort precision, recall, F1 scores, and class counts in descending order based on F1 scores
sorted_indices = np.argsort(f1)[::-1]
precision_sorted = precision[sorted_indices]
recall_sorted = recall[sorted_indices]
f1_sorted = f1[sorted_indices]
classes_sorted = classes[sorted_indices]
class_counts_sorted = class_counts[sorted_indices]

# Calculate mean IoU score for each class
class_iou_scores = []

for class_label in classes_sorted:
    class_indices = np.where(test_labels == class_label)[0]
    class_pred_boxes = pred_boxes[class_indices]
    class_test_boxes = test_boxes[class_indices]

    iou_scores = []

    for i in range(len(class_indices)):
        iou_score = calculate_iou(class_test_boxes[i], class_pred_boxes[i])
        iou_scores.append(iou_score)

    mean_iou_score = np.mean(iou_scores)

    class_iou_scores.append(mean_iou_score)

# Sort the mean IoU scores from best to worst
class_iou_scores_sorted = np.array(class_iou_scores)[sorted_indices]

# Create a DataFrame to store the results
results_df_3 = pd.DataFrame({
    'Class': classes_sorted,
    'F1 Score': f1_sorted,
    'Mean IoU': class_iou_scores_sorted,
    'Class Count': class_counts_sorted
})

# Save the results to a CSV file
results_df_3.to_csv('results_3.csv', index=False)

# Display the DataFrame
print(results_df_3)

In [ ]:
results_df_3

In [ ]:
import shutil

source_file = "/content/results_3.csv"
destination_folder = "/content/drive/MyDrive/project_mark1/"

shutil.copy(source_file, destination_folder)

In [ ]:
source_file = "/content/results_2.csv"
destination_folder = "/content/drive/MyDrive/project_mark1/"

shutil.copy(source_file, destination_folder)

In [ ]:
source_file = "/content/results_1.csv"
destination_folder = "/content/drive/MyDrive/project_mark1/"

shutil.copy(source_file, destination_folder)

In [ ]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('results_3.csv')

# Multiply the count column by 5
df['Class Count'] = df['Class Count'] * 5

# Define the image count ranges
ranges = [
    (0, 100),
    (100, 250),
    (250, 450),
    (450, float('inf'))
]

# Create a mapping for the summarized classes
class_mapping = {
    0: 'Class 0-100',
    1: 'Class 100-250',
    2: 'Class 250-450',
    3: 'Class 450+'
}

# Create an empty DataFrame for the new CSV
new_df = pd.DataFrame(columns=['Class', 'Image Count Range', 'Mean IoU', 'Mean F1', 'Total Images'])

# Calculate the mean scores and total images for each summarized class and image count range
for i, (start, end) in enumerate(ranges):
    filtered_df = df[(df['Class Count'] >= start) & (df['Class Count'] < end)]
    mean_iou = filtered_df['Mean IoU'].mean()
    mean_f1 = filtered_df['F1 Score'].mean()
    total_images = filtered_df['Class Count'].sum()
    new_df.loc[i] = [class_mapping[i], f'{start} - {end}', mean_iou, mean_f1, total_images]

# Save the new DataFrame to a CSV file
new_df.to_csv('outputs_3_iou.csv', index=False)


In [ ]:
new_df.head()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Read the CSV file
df = pd.read_csv('outputs_3_iou.csv')

# Set the plot style using Seaborn
sns.set(style='whitegrid')

# Set the color palette
colors = ['#6C8EBF', '#A1C1D9']

# Create a figure and axes
fig, ax = plt.subplots(figsize=(12, 14))

# Plot the mean IoU and mean F1 scores
df.plot(x='Class', y=['Mean IoU', 'Mean F1'], kind='bar', ax=ax, rot=0, legend=True, color=colors)

# Set the title and axis labels
ax.set_title('Mean IoU and Mean F1 Scores by Class (Top then Full Training // Loss = "IoU")', fontsize=16, fontweight='bold')
ax.set_xlabel('Class', fontsize=14)
ax.set_ylabel('Score', fontsize=12)

# Customize the tick labels and font size
ax.tick_params(axis='x', labelrotation=45, labelsize=10)
ax.tick_params(axis='y', labelsize=10)

# Set the legend
ax.legend(['Mean IoU', 'Mean F1'], loc='upper right', fontsize=10)

# Add data labels to the bars
for p in ax.patches:
    x = p.get_x() + p.get_width() / 2.
    y = p.get_height()
    ax.annotate(f'{y:.3f}', (x, y), ha='center', va='center', xytext=(0, 5), textcoords='offset points', fontsize=10, fontweight='bold')

# Remove the top and right spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add a horizontal grid
ax.yaxis.grid(True, linestyle='dashed')

# Adjust the layout
plt.tight_layout()

# Show the plot
plt.show()



In [ ]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('results_2.csv')

# Multiply the count column by 5
df['Class Count'] = df['Class Count'] * 5

# Define the image count ranges
ranges = [
    (0, 100),
    (100, 250),
    (250, 450),
    (450, float('inf'))
]

# Create a mapping for the summarized classes
class_mapping = {
    0: 'Class 0-100',
    1: 'Class 100-250',
    2: 'Class 250-450',
    3: 'Class 450+'
}

# Create an empty DataFrame for the new CSV
new_df = pd.DataFrame(columns=['Class', 'Image Count Range', 'Mean IoU', 'Mean F1', 'Total Images'])

# Calculate the mean scores and total images for each summarized class and image count range
for i, (start, end) in enumerate(ranges):
    filtered_df = df[(df['Class Count'] >= start) & (df['Class Count'] < end)]
    mean_iou = filtered_df['Mean IoU'].mean()
    mean_f1 = filtered_df['F1 Score'].mean()
    total_images = filtered_df['Class Count'].sum()
    new_df.loc[i] = [class_mapping[i], f'{start} - {end}', mean_iou, mean_f1, total_images]

# Save the new DataFrame to a CSV file
new_df.to_csv('outputs_2_iou.csv', index=False)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Read the CSV file
df = pd.read_csv('outputs_2_iou.csv')

# Set the plot style using Seaborn
sns.set(style='whitegrid')

# Set the color palette
colors = ['#6C8EBF', '#A1C1D9']

# Create a figure and axes
fig, ax = plt.subplots(figsize=(12, 14))

# Plot the mean IoU and mean F1 scores
df.plot(x='Class', y=['Mean IoU', 'Mean F1'], kind='bar', ax=ax, rot=0, legend=True, color=colors)

# Set the title and axis labels
ax.set_title('Mean IoU and Mean F1 Scores by Class (Top-only Training // Loss = "IoU")', fontsize=16, fontweight='bold')
ax.set_xlabel('Class', fontsize=14)
ax.set_ylabel('Score', fontsize=12)

# Customize the tick labels and font size
ax.tick_params(axis='x', labelrotation=45, labelsize=10)
ax.tick_params(axis='y', labelsize=10)

# Set the legend
ax.legend(['Mean IoU', 'Mean F1'], loc='upper right', fontsize=10)

# Add data labels to the bars
for p in ax.patches:
    x = p.get_x() + p.get_width() / 2.
    y = p.get_height()
    ax.annotate(f'{y:.3f}', (x, y), ha='center', va='center', xytext=(0, 5), textcoords='offset points', fontsize=10, fontweight='bold')

# Remove the top and right spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add a horizontal grid
ax.yaxis.grid(True, linestyle='dashed')

# Adjust the layout
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('results_1.csv')

# Multiply the count column by 5
df['Class Count'] = df['Class Count'] * 5

# Define the image count ranges
ranges = [
    (0, 100),
    (100, 250),
    (250, 450),
    (450, float('inf'))
]

# Create a mapping for the summarized classes
class_mapping = {
    0: 'Class 0-100',
    1: 'Class 100-250',
    2: 'Class 250-450',
    3: 'Class 450+'
}

# Create an empty DataFrame for the new CSV
new_df = pd.DataFrame(columns=['Class', 'Image Count Range', 'Mean IoU', 'Mean F1', 'Total Images'])

# Calculate the mean scores and total images for each summarized class and image count range
for i, (start, end) in enumerate(ranges):
    filtered_df = df[(df['Class Count'] >= start) & (df['Class Count'] < end)]
    mean_iou = filtered_df['Mean IoU'].mean()
    mean_f1 = filtered_df['F1 Score'].mean()
    total_images = filtered_df['Class Count'].sum()
    new_df.loc[i] = [class_mapping[i], f'{start} - {end}', mean_iou, mean_f1, total_images]

# Save the new DataFrame to a CSV file
new_df.to_csv('outputs_1_iou.csv', index=False)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Read the CSV file
df = pd.read_csv('outputs_1_iou.csv')

# Set the plot style using Seaborn
sns.set(style='whitegrid')

# Set the color palette
colors = ['#6C8EBF', '#A1C1D9']

# Create a figure and axes
fig, ax = plt.subplots(figsize=(12, 14))

# Plot the mean IoU and mean F1 scores
df.plot(x='Class', y=['Mean IoU', 'Mean F1'], kind='bar', ax=ax, rot=0, legend=True, color=colors)

# Set the title and axis labels
ax.set_title('Mean IoU and Mean F1 Scores by Class (Full Training // Loss = "IoU")', fontsize=16, fontweight='bold')
ax.set_xlabel('Class', fontsize=14)
ax.set_ylabel('Score', fontsize=12)

# Customize the tick labels and font size
ax.tick_params(axis='x', labelrotation=45, labelsize=10)
ax.tick_params(axis='y', labelsize=10)

# Set the legend
ax.legend(['Mean IoU', 'Mean F1'], loc='upper right', fontsize=10)

# Add data labels to the bars
for p in ax.patches:
    x = p.get_x() + p.get_width() / 2.
    y = p.get_height()
    ax.annotate(f'{y:.3f}', (x, y), ha='center', va='center', xytext=(0, 5), textcoords='offset points', fontsize=10, fontweight='bold')

# Remove the top and right spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add a horizontal grid
ax.yaxis.grid(True, linestyle='dashed')

# Adjust the layout
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
import shutil

source_file = "/content/outputs_3_iou.csv"
destination_folder = "/content/drive/MyDrive/project_mark1/"

shutil.copy(source_file, destination_folder)

In [ ]:
import shutil

source_file = "/content/outputs_2_iou.csv"
destination_folder = "/content/drive/MyDrive/project_mark1/"

shutil.copy(source_file, destination_folder)

In [ ]:
import shutil

source_file = "/content/outputs_1_iou.csv"
destination_folder = "/content/drive/MyDrive/project_mark1/"

shutil.copy(source_file, destination_folder)

In [ ]:
# # Save test_carside_df to CSV
# test_carside_df.to_csv(os.path.join(parent_dir, "test_carside_df.csv"), index=False)

# # Save val_carside_df to CSV
# val_carside_df.to_csv(os.path.join(parent_dir, "val_carside_df.csv"), index=False)

# # Save carside_df to CSV
# carside_df.to_csv(os.path.join(parent_dir, "carside_df.csv"), index=False)


In [ ]:
column_names = ['Image Path', 'Class Name', 'Image Name', 'Bounding Box', 'Height', 'Width', 'label',
                'normalized', 'Xmin', 'Ymin', 'Xmax', 'Ymax']


test_carside_df.columns = column_names
val_carside_df.columns = column_names
carside_df.columns = column_names

In [ ]:
def load_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))
    image = image.astype(np.float32) / 255.0
    return image

In [ ]:
def plot_images_with_bboxes(df):
    fig, axes = plt.subplots(1, 5, figsize=(20, 4))
    for i, row in df.head(5).iterrows():
        image_path = row['Image Path']
        class_name = row['Class Name']
        image_name = row['Image Name']
        xmin, ymin, xmax, ymax = row['Xmin']*224, row['Ymin']*224, row['Xmax']*224, row['Ymax']*224

        image = load_image(image_path)

        ax = axes[i]
        ax.imshow(image)
        ax.axis('off')

        # Add bounding box
        bbox = patches.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(bbox)

        ax.set_title(f'Class: {class_name}\nImage: {image_name}')

    plt.show()

# Plot images with bounding boxes for val_df, test_df, train_df
plot_images_with_bboxes(val_carside_df)
plot_images_with_bboxes(test_carside_df)
plot_images_with_bboxes(carside_df)

In [ ]:
####################################################
############## MOST IMPORTANT PART #################
####################################################
def resize_image(image, target_size):
    return cv2.resize(image, target_size)

def plot_images(images, boxes, labels, dataset_name):
    fig, axes = plt.subplots(1, len(images), figsize=(12, 4))
    for i, ax in enumerate(axes):
        ax.imshow(images[i])
        xmin, ymin, xmax, ymax = boxes[i]
        ax.add_patch(plt.Rectangle((xmin*224, ymin*224), xmax*224 - xmin*224, ymax*224 - ymin*224, fill=False, color='red', linewidth=2))
        ax.set_title(labels[i])
        ax.axis('off')
    plt.suptitle(f'{dataset_name} Images with Bounding Boxes')
    plt.show()

# Append data from val_df into arrays
val_images = []
val_boxes = []
val_labels = []

for index, row in val_carside_df.iterrows():
    image_path = row['Image Path']
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = resize_image(image, (224, 224))
    val_images.append(image)

    xmin, ymin, xmax, ymax = map(float, [row['Xmin'], row['Ymin'], row['Xmax'], row['Ymax']])
    val_boxes.append([xmin, ymin, xmax, ymax])

    label = row['Class Name']
    val_labels.append(label)

# Convert the lists to NumPy arrays
val_images = np.array(val_images, dtype='float32')
val_boxes = np.array(val_boxes, dtype='float32')
val_labels = np.array(val_labels)

# Normalize the images
val_images = val_images / 255.0

# Plot the images, bounding boxes, and class names for the first 4 images from the val_df dataset
plot_images(val_images[:5], val_boxes[:5], val_labels[:5], 'Validation')

# Append data from train_df into arrays
train_images = []
train_boxes = []
train_labels = []

for index, row in carside_df.iterrows():
    image_path = row['Image Path']
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = resize_image(image, (224, 224))
    train_images.append(image)

    xmin, ymin, xmax, ymax = map(float, [row['Xmin'], row['Ymin'], row['Xmax'], row['Ymax']])
    train_boxes.append([xmin, ymin, xmax, ymax])

    label = row['Class Name']
    train_labels.append(label)

# Convert the lists to NumPy arrays
train_images = np.array(train_images, dtype='float32')
train_boxes = np.array(train_boxes, dtype='float32')
train_labels = np.array(train_labels)

# Normalize the images
train_images = train_images / 255.0

# Plot the images, bounding boxes, and class names for the first 4 images from the train_df dataset
plot_images(train_images[:5], train_boxes[:5], train_labels[:5], 'Train')

# Append data from test_df into arrays
test_images = []
test_boxes = []
test_labels = []

for index, row in test_carside_df.iterrows():
    image_path = row['Image Path']
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = resize_image(image, (224, 224))
    test_images.append(image)

    xmin, ymin, xmax, ymax = map(float, [row['Xmin'], row['Ymin'], row['Xmax'], row['Ymax']])
    test_boxes.append([xmin, ymin, xmax, ymax])

    label = row['Class Name']
    test_labels.append(label)

# Convert the lists to NumPy arrays
test_images = np.array(test_images, dtype='float32')
test_boxes = np.array(test_boxes, dtype='float32')
test_labels = np.array(test_labels)

# Normalize the images
test_images = test_images / 255.0

# Plot the images, bounding boxes, and class names for the first 4 images from the test_df dataset
plot_images(test_images[:5], test_boxes[:5], test_labels[:5], 'Test')

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Perform label encoding on the target labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
val_labels_encoded = label_encoder.transform(val_labels)

# Convert the encoded labels to one-hot encoding
train_labels_onehot = to_categorical(train_labels_encoded, num_classes=1) ## ==> update number of classes
val_labels_onehot = to_categorical(val_labels_encoded, num_classes=1) ## ==> update number of classes

# Model architecture
input_layer = Input(shape=(224, 224, 3))
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=input_layer)


flatten = Flatten()(base_model.output)


bboxHead = Dense(1024, activation="relu")(flatten)

bboxHead = Dense(512, activation="relu")(bboxHead)

bboxHead = Dense(256, activation="relu")(bboxHead)

bboxHead = Dense(128, activation="relu")(bboxHead)

bboxHead = Dense(64, activation="relu")(bboxHead)

bboxHead = Dense(32, activation="relu")(bboxHead)

bbox_output = Dense(4, activation='linear', name='bounding_box')(bboxHead)






carside_resnet_mse_1 = Model(inputs=input_layer, outputs=[bbox_output])



# Compile the model
carside_resnet_mse_1.compile(optimizer='adam', loss="mse")
carside_resnet_mse_1.summary()

In [ ]:
from keras.callbacks import EarlyStopping

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=7)


# Train the model with early stopping
carside_resnet_mse_1_history = carside_resnet_mse_1.fit(
    train_images,
    {'bounding_box': train_boxes},  # Update the label key to 'bounding_box'
    validation_data=(val_images, {'bounding_box': val_boxes}),  # Update the label key to 'bounding_box'
    epochs=300,
    batch_size=32,
    callbacks=[callback]
)

In [ ]:
# Save the label encoder
with open('/content/drive/MyDrive/project_mark1/carside_resnet_mse_1_label_encoder.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)

# Save the entire model to a HDF5 file
carside_resnet_mse_1.save('/content/drive/MyDrive/project_mark1/carside_resnet_mse_1_history.h5')

# Save the history object to a file using pickle
with open('/content/drive/MyDrive/project_mark1/model_fine_tuned_history.pkl', 'wb') as file:
    pickle.dump(carside_resnet_mse_1_history.history, file)

In [ ]:
import matplotlib.pyplot as plt
import pickle
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load model history from file
history_path = '/content/drive/MyDrive/project_mark1/model_resnet_mse_1_history.pkl'
with open(history_path, 'rb') as f:
    model_resnet_mse_1_history = pickle.load(f)

# Print the available keys in the model history
print(model_resnet_mse_1_history.keys())

# Plot the losses and metrics
plt.figure(figsize=(12, 10))
plt.style.use('seaborn-whitegrid')

# Classification Loss
plt.subplot(2, 2, 1)
plt.plot(model_resnet_mse_1_history['class_output_loss'], label='Train Classification Loss', color='blue')
plt.plot(model_resnet_mse_1_history['val_class_output_loss'], label='Validation Classification Loss', color='orange')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Classification Loss', fontsize=12)
plt.legend()

# Bounding Box Loss
plt.subplot(2, 2, 2)
plt.plot(model_resnet_mse_1_history['bounding_box_loss'], label='Bounding Box Loss', color='blue')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Bounding Box Loss', fontsize=12)
plt.legend()

# Accuracy
if 'class_output_accuracy' in model_resnet_mse_1_history:
    plt.subplot(2, 2, 3)
    plt.plot(model_resnet_mse_1_history['class_output_accuracy'], label='Train Accuracy', color='blue')
    plt.plot(model_resnet_mse_1_history['val_class_output_accuracy'], label='Validation Accuracy', color='orange')
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('Accuracy', fontsize=12)
    plt.title('Classification Accuracy', fontsize=12)
    plt.legend()

# Validation Loss
plt.subplot(2, 2, 4)
plt.plot(model_resnet_mse_1_history['val_loss'], label='Validation Loss', color='blue')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Validation Loss', fontsize=12)
plt.legend()

# Adjust layout and spacing
plt.tight_layout()

# Show the plots
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pickle
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load model history from file
history_path = '/content/drive/MyDrive/project_mark1/model_resnet_mse_1_history.pkl'
with open(history_path, 'rb') as f:
    model_resnet_mse_1_history = pickle.load(f)

# Print the available keys in the model history
print(model_resnet_mse_1_history.keys())

# Plot Classification Loss
plt.figure(figsize=(10, 5))
plt.plot(model_resnet_mse_1_history['class_output_loss'], label='Train Classification Loss', color='blue')
plt.plot(model_resnet_mse_1_history['val_class_output_loss'], label='Validation Classification Loss', color='orange')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Classification Loss - (Full Training(IoU))', fontsize=12)
plt.legend()

# Find and annotate best and worst values
best_class_loss = min(model_resnet_mse_1_history['class_output_loss'])
worst_class_loss = max(model_resnet_mse_1_history['class_output_loss'])
plt.annotate(f'Best: {best_class_loss:.4f}', xy=(model_resnet_mse_1_history['class_output_loss'].index(best_class_loss), best_class_loss), xytext=(10, 5),
             textcoords='offset points', color='blue', fontsize=10)
plt.annotate(f'Worst: {worst_class_loss:.4f}', xy=(model_resnet_mse_1_history['class_output_loss'].index(worst_class_loss), worst_class_loss),
             xytext=(-60, -15), textcoords='offset points', color='blue', fontsize=10)

# Show Classification Loss plot
plt.tight_layout()
plt.show()

# Plot Bounding Box Loss and Validation Loss
plt.figure(figsize=(10, 5))
plt.plot(model_resnet_mse_1_history['bounding_box_loss'], label='Bounding Box Loss', color='blue')
plt.plot(model_resnet_mse_1_history['val_loss'], label='Validation Loss', color='orange')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Bounding Box and Validation Loss - (Full Training(IoU)', fontsize=12)
plt.legend()

# Find and annotate best and worst values
best_bbox_loss = min(model_resnet_mse_1_history['bounding_box_loss'])
worst_bbox_loss = max(model_resnet_mse_1_history['bounding_box_loss'])
best_val_loss = min(model_resnet_mse_1_history['val_loss'])
worst_val_loss = max(model_resnet_mse_1_history['val_loss'])
plt.annotate(f'Best BBox: {best_bbox_loss:.4f}', xy=(model_resnet_mse_1_history['bounding_box_loss'].index(best_bbox_loss), best_bbox_loss), xytext=(10, 5),
             textcoords='offset points', color='blue', fontsize=10)
plt.annotate(f'Worst BBox: {worst_bbox_loss:.4f}', xy=(model_resnet_mse_1_history['bounding_box_loss'].index(worst_bbox_loss), worst_bbox_loss),
             xytext=(-60, -15), textcoords='offset points', color='blue', fontsize=10)
plt.annotate(f'Best Val: {best_val_loss:.4f}', xy=(model_resnet_mse_1_history['val_loss'].index(best_val_loss), best_val_loss), xytext=(10, 5),
             textcoords='offset points', color='orange', fontsize=10)
plt.annotate(f'Worst Val: {worst_val_loss:.4f}', xy=(model_resnet_mse_1_history['val_loss'].index(worst_val_loss), worst_val_loss),
             xytext=(-60, -15), textcoords='offset points', color='orange', fontsize=10)

# Show Bounding Box and Validation Loss plot
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pickle
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load model history from file
history_path = '/content/drive/MyDrive/project_mark1/model_resnet_mse_2_history.pkl'
with open(history_path, 'rb') as f:
    model_resnet_mse_1_history = pickle.load(f)

# Print the available keys in the model history
print(model_resnet_mse_1_history.keys())

# Plot Classification Loss
plt.figure(figsize=(10, 5))
plt.plot(model_resnet_mse_1_history['class_output_loss'], label='Train Classification Loss', color='blue')
plt.plot(model_resnet_mse_1_history['val_class_output_loss'], label='Validation Classification Loss', color='orange')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Classification Loss - (Top Only(IoU))', fontsize=12)
plt.legend()

# Find and annotate best and worst values
best_class_loss = min(model_resnet_mse_1_history['class_output_loss'])
worst_class_loss = max(model_resnet_mse_1_history['class_output_loss'])
plt.annotate(f'Best: {best_class_loss:.4f}', xy=(model_resnet_mse_1_history['class_output_loss'].index(best_class_loss), best_class_loss), xytext=(10, 5),
             textcoords='offset points', color='blue', fontsize=10)
plt.annotate(f'Worst: {worst_class_loss:.4f}', xy=(model_resnet_mse_1_history['class_output_loss'].index(worst_class_loss), worst_class_loss),
             xytext=(-60, -15), textcoords='offset points', color='blue', fontsize=10)

# Show Classification Loss plot
plt.tight_layout()
plt.show()

# Plot Bounding Box Loss and Validation Loss
plt.figure(figsize=(10, 5))
plt.plot(model_resnet_mse_1_history['bounding_box_loss'], label='Bounding Box Loss', color='blue')
plt.plot(model_resnet_mse_1_history['val_loss'], label='Validation Loss', color='orange')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Bounding Box and Validation Loss - (Top-only(IoU)', fontsize=12)
plt.legend()

# Find and annotate best and worst values
best_bbox_loss = min(model_resnet_mse_1_history['bounding_box_loss'])
worst_bbox_loss = max(model_resnet_mse_1_history['bounding_box_loss'])
best_val_loss = min(model_resnet_mse_1_history['val_loss'])
worst_val_loss = max(model_resnet_mse_1_history['val_loss'])
plt.annotate(f'Best BBox: {best_bbox_loss:.4f}', xy=(model_resnet_mse_1_history['bounding_box_loss'].index(best_bbox_loss), best_bbox_loss), xytext=(10, 5),
             textcoords='offset points', color='blue', fontsize=10)
plt.annotate(f'Worst BBox: {worst_bbox_loss:.4f}', xy=(model_resnet_mse_1_history['bounding_box_loss'].index(worst_bbox_loss), worst_bbox_loss),
             xytext=(-60, -15), textcoords='offset points', color='blue', fontsize=10)
plt.annotate(f'Best Val: {best_val_loss:.4f}', xy=(model_resnet_mse_1_history['val_loss'].index(best_val_loss), best_val_loss), xytext=(10, 5),
             textcoords='offset points', color='orange', fontsize=10)
plt.annotate(f'Worst Val: {worst_val_loss:.4f}', xy=(model_resnet_mse_1_history['val_loss'].index(worst_val_loss), worst_val_loss),
             xytext=(-60, -15), textcoords='offset points', color='orange', fontsize=10)

# Show Bounding Box and Validation Loss plot
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pickle
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load model history from file
history_path = '/content/drive/MyDrive/project_mark1/model_fine_tuned_history.pkl'
with open(history_path, 'rb') as f:
    model_resnet_mse_1_history = pickle.load(f)

# Print the available keys in the model history
print(model_resnet_mse_1_history.keys())

# Plot Classification Loss
plt.figure(figsize=(10, 5))
plt.plot(model_resnet_mse_1_history['class_output_loss'], label='Train Classification Loss', color='blue')
plt.plot(model_resnet_mse_1_history['val_class_output_loss'], label='Validation Classification Loss', color='orange')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Classification Loss - (Top then Full(IoU))', fontsize=12)
plt.legend()

# Find and annotate best and worst values
best_class_loss = min(model_resnet_mse_1_history['class_output_loss'])
worst_class_loss = max(model_resnet_mse_1_history['class_output_loss'])
plt.annotate(f'Best: {best_class_loss:.4f}', xy=(model_resnet_mse_1_history['class_output_loss'].index(best_class_loss), best_class_loss), xytext=(10, 5),
             textcoords='offset points', color='blue', fontsize=10)
plt.annotate(f'Worst: {worst_class_loss:.4f}', xy=(model_resnet_mse_1_history['class_output_loss'].index(worst_class_loss), worst_class_loss),
             xytext=(-60, -15), textcoords='offset points', color='blue', fontsize=10)

# Show Classification Loss plot
plt.tight_layout()
plt.show()

# Plot Bounding Box Loss and Validation Loss
plt.figure(figsize=(10, 5))
plt.plot(model_resnet_mse_1_history['bounding_box_loss'], label='Bounding Box Loss', color='blue')
plt.plot(model_resnet_mse_1_history['val_loss'], label='Validation Loss', color='orange')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Bounding Box and Validation Loss - (Top then Full(IoU)', fontsize=12)
plt.legend()

# Find and annotate best and worst values
best_bbox_loss = min(model_resnet_mse_1_history['bounding_box_loss'])
worst_bbox_loss = max(model_resnet_mse_1_history['bounding_box_loss'])
best_val_loss = min(model_resnet_mse_1_history['val_loss'])
worst_val_loss = max(model_resnet_mse_1_history['val_loss'])
plt.annotate(f'Best BBox: {best_bbox_loss:.4f}', xy=(model_resnet_mse_1_history['bounding_box_loss'].index(best_bbox_loss), best_bbox_loss), xytext=(10, 5),
             textcoords='offset points', color='blue', fontsize=10)
plt.annotate(f'Worst BBox: {worst_bbox_loss:.4f}', xy=(model_resnet_mse_1_history['bounding_box_loss'].index(worst_bbox_loss), worst_bbox_loss),
             xytext=(-60, -15), textcoords='offset points', color='blue', fontsize=10)
plt.annotate(f'Best Val: {best_val_loss:.4f}', xy=(model_resnet_mse_1_history['val_loss'].index(best_val_loss), best_val_loss), xytext=(10, 5),
             textcoords='offset points', color='orange', fontsize=10)
plt.annotate(f'Worst Val: {worst_val_loss:.4f}', xy=(model_resnet_mse_1_history['val_loss'].index(worst_val_loss), worst_val_loss),
             xytext=(-60, -15), textcoords='offset points', color='orange', fontsize=10)

# Show Bounding Box and Validation Loss plot
plt.tight_layout()
plt.show()

In [ ]:
from tensorflow.keras.utils import plot_model

# Assuming you have already loaded your model
model = load_model('/content/drive/MyDrive/project_mark1/model_top_layers.h5')

# Plot the model architecture
plot_model(model, to_file='model_architecture.png', show_shapes=True)

In [ ]:
!pip install visualkeras
import visualkeras
visualkeras.layered_view(model)

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.applications import ResNet50

In [ ]:
### --> 0.42
### --> only carside --> imagenet --> mse --> Reset50

In [ ]:

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Perform label encoding on the target labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
val_labels_encoded = label_encoder.transform(val_labels)

# Convert the encoded labels to one-hot encoding
train_labels_onehot = to_categorical(train_labels_encoded, num_classes=1) ## ==> update number of classes
val_labels_onehot = to_categorical(val_labels_encoded, num_classes=1) ## ==> update number of classes

# Model architecture
input_layer = Input(shape=(224, 224, 3))
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=input_layer)

flatten = Flatten()(base_model.output)

bboxHead = Dense(1024, activation="relu")(flatten)
bboxHead = Dense(512, activation="relu")(bboxHead)
bboxHead = Dense(256, activation="relu")(bboxHead)
bboxHead = Dense(128, activation="relu")(bboxHead)
bboxHead = Dense(64, activation="relu")(bboxHead)
bboxHead = Dense(32, activation="relu")(bboxHead)
bbox_output = Dense(4, activation='linear', name='bounding_box')(bboxHead)

carside_resnet_mse_1 = Model(inputs=input_layer, outputs=[bbox_output])

# Compile the model
carside_resnet_mse_1.compile(optimizer='adam', loss="mse")
carside_resnet_mse_1.summary()

In [ ]:
from keras.callbacks import EarlyStopping
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [ ]:
# Train the model with early stopping
carside_resnet_mse_1_history = carside_resnet_mse_1.fit(
    train_images,
    {'bounding_box': train_boxes},  # Update the label key to 'bounding_box'
    validation_data=(val_images, {'bounding_box': val_boxes}),  # Update the label key to 'bounding_box'
    epochs=300,
    batch_size=32,
    callbacks=[callback]
)

In [ ]:
# Save the label encoder
with open('/content/drive/MyDrive/project_mark1/carside_resnet_mse_1_label_encoder.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)

# Save the entire model to a HDF5 file
carside_resnet_mse_1.save('/content/drive/MyDrive/project_mark1/carside_resnet_mse_1_history.h5')

# Save the history object to a file using pickle
with open('/content/drive/MyDrive/project_mark1/model_fine_tuned_history.pkl', 'wb') as file:
    pickle.dump(carside_resnet_mse_1_history.history, file)

In [ ]:
import matplotlib.pyplot as plt

# Get the loss values from the history object
train_loss = carside_resnet_mse_1_history.history['loss']
val_loss = carside_resnet_mse_1_history.history['val_loss']

# Plot the loss values
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Loss Curve')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
best_train_loss = min(train_loss)
best_val_loss = min(val_loss)
worst_train_loss = max(train_loss)
worst_val_loss = max(val_loss)

print("Best Training Loss:", best_train_loss)
print("Best Validation Loss:", best_val_loss)
print("Worst Training Loss:", worst_train_loss)
print("Worst Validation Loss:", worst_val_loss)

In [ ]:
import matplotlib.pyplot as plt

# Get the loss and accuracy values from the history object
train_loss = carside_resnet_mse_1_history.history['loss']
val_loss = carside_resnet_mse_1_history.history['val_loss']
train_accuracy = carside_resnet_mse_1_history.history['accuracy']
val_accuracy = carside_resnet_mse_1_history.history['val_accuracy']

# Create subplots for loss and accuracy
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 8))

# Plot loss values
ax1.plot(train_loss, label='Training Loss')
ax1.plot(val_loss, label='Validation Loss')
ax1.set_title('Loss')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss')
ax1.legend()
ax1.grid(True)

# Plot accuracy values
ax2.plot(train_accuracy, label='Training Accuracy')
ax2.plot(val_accuracy, label='Validation Accuracy')
ax2.set_title('Accuracy')
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Accuracy')
ax2.legend()
ax2.grid(True)

# Adjust spacing between subplots
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
def calculate_iou(box1, box2):
    # Extract coordinates from the bounding boxes
    x1_min, y1_min, x1_max, y1_max = box1
    x2_min, y2_min, x2_max, y2_max = box2

    # Calculate the intersection area
    intersection_area = max(0, min(x1_max, x2_max*19) - max(x1_min, x2_min*19)) * max(0, min(y1_max, y2_max*30) - max(y1_min, y2_min*30))

    # Calculate the union area
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max*19 - x2_min*19) * (y2_max*30 - y2_min*30)
    union_area = box1_area + box2_area - intersection_area

    # Calculate the IoU
    iou = intersection_area / union_area

    return iou

In [ ]:
# Make predictions on the test images
predictions = carside_resnet_mse_1.predict(test_images)

# Calculate IoU for each image
iou_scores = []
for i in range(len(predictions)):
    pred_box = predictions[i]
    true_box = test_boxes[i]
    iou = calculate_iou(pred_box, true_box)
    iou_scores.append(iou)

# Display predicted and original bounding box values with IoU scores
for i in range(len(predictions)):
    print("Image", i+1)
    print("Predicted Bounding Box:", predictions[i])
    print("Original Bounding Box:", test_boxes[i])

    print("IoU Score:", iou_scores[i])
    print()

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Load the saved model
model_top = load_model('/content/drive/MyDrive/project_mark2/model_resnet_mse_1.h5')


In [ ]:
# Model architecture
input_layer = Input(shape=(224, 224, 3))
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=input_layer)

In [ ]:
flatten = Flatten()(base_model.output)

bboxHead = Dense(1024, activation="relu")(flatten)
bboxHead = Dense(512, activation="relu")(bboxHead)



bbox_output = Dense(4, activation='linear', name='bounding_box')(bboxHead)

model = Model(inputs=input_layer, outputs=bbox_output)


In [ ]:
# Load the saved weights layer by layer
for layer in model.layers:
    if layer.name in model_top.layers:
        layer.set_weights(model_top.get_layer(layer.name).get_weights())

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

In [ ]:
# Define early stopping callback

# Train the model with early stopping
history = model.fit(train_images, train_boxes,
                    validation_data=(val_images, val_boxes),
                    epochs=300, batch_size=8)

In [ ]:
# Save the trained model
model.save('/content/drive/MyDrive/trained_bbox_model_2.h5')

In [ ]:
import matplotlib.pyplot as plt

# Get the training and validation loss values from the history object
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Get the epoch with the best and worst training loss
best_train_epoch = train_loss.index(min(train_loss)) + 1
worst_train_epoch = train_loss.index(max(train_loss)) + 1

# Get the epoch with the best and worst validation loss
best_val_epoch = val_loss.index(min(val_loss)) + 1
worst_val_epoch = val_loss.index(max(val_loss)) + 1

# Plot the training and validation losses
epochs = range(1, len(train_loss) + 1)
plt.plot(epochs, train_loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot the best and worst values
plt.plot(best_train_epoch, min(train_loss), 'go', label='Best Training Loss')
plt.plot(worst_train_epoch, max(train_loss), 'ro', label='Worst Training Loss')
plt.plot(best_val_epoch, min(val_loss), 'g^', label='Best Validation Loss')
plt.plot(worst_val_epoch, max(val_loss), 'r^', label='Worst Validation Loss')

plt.annotate(f'({best_train_epoch}, {min(train_loss):.4f})',
             xy=(best_train_epoch, min(train_loss)),
             xytext=(best_train_epoch, min(train_loss) + 0.5),
             arrowprops=dict(facecolor='green', arrowstyle='->'))

plt.annotate(f'({worst_train_epoch}, {max(train_loss):.4f})',
             xy=(worst_train_epoch, max(train_loss)),
             xytext=(worst_train_epoch, max(train_loss) + 0.5),
             arrowprops=dict(facecolor='red', arrowstyle='->'))

plt.annotate(f'({best_val_epoch}, {min(val_loss):.4f})',
             xy=(best_val_epoch, min(val_loss)),
             xytext=(best_val_epoch, min(val_loss) + 0.5),
             arrowprops=dict(facecolor='green', arrowstyle='->'))

plt.annotate(f'({worst_val_epoch}, {max(val_loss):.4f})',
             xy=(worst_val_epoch, max(val_loss)),
             xytext=(worst_val_epoch, max(val_loss) + 0.5),
             arrowprops=dict(facecolor='red', arrowstyle='->'))

plt.show()

In [ ]:
test_boxes

In [ ]:
test_images

In [ ]:
# Perform predictions
predicted_boxes = model.predict(test_images)

# Calculate IoU scores for each image
iou_scores = []
for pred_box, true_box in zip(predicted_boxes, test_boxes):
    iou_score = calculate_iou(pred_box, true_box)
    iou_scores.append(iou_score)

# Multiply bounding boxes
multiplied_boxes = np.multiply(predicted_boxes, test_boxes)

# Denormalize bounding boxes for plotting
def denormalize_boxes(boxes, image_shape):
    height, width = image_shape[:2]
    boxes = np.array(boxes)
    boxes[:, [0, 2]] *= width
    boxes[:, [1, 3]] *= height
    return boxes

# Plot images with predicted and original bounding boxes
def plot_image_with_boxes(image, boxes, title):
    # Create figure and axes
    fig, ax = plt.subplots(1)
    ax.imshow(image)

    # Plot bounding boxes
    for box in boxes:
        xmin, ymin, xmax, ymax = box
        rect = plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, fill=False, edgecolor='r', linewidth=2)
        ax.add_patch(rect)

    # Set title
    ax.set_title(title)

    # Show the plot
    plt.show()

# Iterate over each image and plot with bounding boxes
for i in range(len(test_images)):
    # Denormalize bounding boxes
    denorm_test_boxes = denormalize_boxes([test_boxes[i]], test_images[i].shape)
    denorm_predicted_boxes = denormalize_boxes([predicted_boxes[i]], test_images[i].shape)

    # Plot the image with original bounding boxes
    plot_image_with_boxes(test_images[i], denorm_test_boxes, "Original Bounding Boxes")

    # Plot the image with predicted bounding boxes
    plot_image_with_boxes(test_images[i], denorm_predicted_boxes, "Predicted Bounding Boxes")

    # Print the IoU score
    print("IoU score for image", i+1, ":", iou_scores[i])

    # Print the multiplied bounding boxes
    print("Multiplied bounding box for image", i+1, ":", multiplied_boxes[i])
    print()

# Calculate the mean IoU score
mean_iou = np.mean(iou_scores)
print("Mean IoU Score:", mean_iou)

In [ ]:

# Perform predictions
predicted_boxes = model.predict(test_images)

# Calculate IoU scores for each image
iou_scores = []
for pred_box, true_box in zip(predicted_boxes, test_boxes):
    iou_score = calculate_iou(pred_box, true_box)
    iou_scores.append(iou_score)

# Print the bounding boxes and IoU scores for each image
for i in range(len(test_images)):
    print("Bounding Box for Image", i+1)
    print("Predicted Box:", predicted_boxes[i])
    print("True Box:", test_boxes[i])
    print("IoU Score:", iou_scores[i])
    print()

# Calculate the mean IoU score
mean_iou = np.mean(iou_scores)
print("Mean IoU Score:", mean_iou)

In [ ]:

# Perform predictions
predicted_boxes = model.predict(test_images)

# Calculate IoU scores for each image
iou_scores = []
for pred_box, true_box in zip(predicted_boxes, test_boxes):
    iou_score = calculate_iou(pred_box, true_box)
    iou_scores.append(iou_score)

# Multiply bounding boxes
multiplied_boxes = np.multiply(predicted_boxes, test_boxes)

# Denormalize bounding boxes for plotting
def denormalize_boxes(boxes, image_shape):
    height, width = image_shape[:2]
    boxes = np.array(boxes)
    boxes[:, [0, 2]] *= width
    boxes[:, [1, 3]] *= height
    return boxes

# Plot images with predicted and original bounding boxes
def plot_image_with_boxes(image, boxes, iou_score, title):
    # Create figure and axes
    fig, ax = plt.subplots(1)
    ax.imshow(image)

    # Plot bounding boxes
    for box in boxes:
        xmin, ymin, xmax, ymax = box
        rect = cv2.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, fill=False, edgecolor='r', linewidth=2)
        ax.add_patch(rect)

    # Set title and IoU text
    ax.set_title(f"{title}\nIoU: {iou_score:.2f}")

    # Show the plot
    plt.show()

# Iterate over each image and plot with bounding boxes and IoU
mean_iou = 0.0
for i in range(len(test_images)):
    # Denormalize bounding boxes
    denorm_test_boxes = denormalize_boxes([test_boxes[i]], test_images[i].shape)
    denorm_predicted_boxes = denormalize_boxes([predicted_boxes[i]], test_images[i].shape)

    # Calculate IoU score
    iou_score = iou_scores[i]
    mean_iou += iou_score

    # Plot the image with original and predicted bounding boxes
    plot_image_with_boxes(test_images[i], denorm_test_boxes, iou_score, "Original and Predicted Bounding Boxes")

# Calculate the mean IoU score
mean_iou /= len(test_images)
print("Mean IoU Score:", mean_iou)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
# Perform predictions
predicted_boxes = model.predict(test_images)

# Calculate IoU scores for each image
iou_scores = []
for pred_box, true_box in zip(predicted_boxes, test_boxes):
    iou_score = calculate_iou(pred_box, true_box)
    iou_scores.append(iou_score)

# Multiply bounding boxes
multiplied_boxes = np.multiply(predicted_boxes, test_boxes)

# Denormalize bounding boxes for plotting
def denormalize_boxes(boxes, image_shape):
    height, width = image_shape[:2]
    boxes = np.array(boxes)
    boxes[:, [0, 2]] *= width
    boxes[:, [1, 3]] *= height
    return boxes

# Plot images with original and predicted bounding boxes
def plot_image_with_boxes(image, true_boxes, pred_boxes, iou_score, title):
    # Create figure and axes
    fig, ax = plt.subplots(1)
    ax.imshow(image)

    # Plot original bounding boxes
    for box in true_boxes:
        xmin, ymin, xmax, ymax = box
        rect = Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, fill=False, edgecolor='r', linewidth=2)
        ax.add_patch(rect)

    # Plot predicted bounding boxes
    for box in pred_boxes:
        xmin, ymin, xmax, ymax = box
        rect = Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, fill=False, edgecolor='b', linewidth=2)
        ax.add_patch(rect)

    # Set title and IoU text
    ax.set_title(f"{title}\nIoU: {iou_score:.2f}")

    # Show the plot
    plt.show()

# Iterate over each image and plot with bounding boxes and IoU
mean_iou = 0.0
for i in range(len(test_images)):
    # Denormalize bounding boxes
    denorm_test_boxes = denormalize_boxes([test_boxes[i]], test_images[i].shape)
    denorm_predicted_boxes = denormalize_boxes([predicted_boxes[i]], test_images[i].shape)

    # Calculate IoU score
    iou_score = iou_scores[i]
    mean_iou += iou_score

    # Plot the image with original and predicted bounding boxes
    plot_image_with_boxes(test_images[i], denorm_test_boxes, denorm_predicted_boxes, iou_score, "Original and Predicted Bounding Boxes")

# Calculate the mean IoU score
mean_iou /= len(test_images)
print("Mean IoU Score:", mean_iou)

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Model architecture
input_layer = Input(shape=(224, 224, 3))
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=input_layer)

In [ ]:
flatten = Flatten()(base_model.output)

bboxHead = Dense(1024, activation="relu")(flatten)
bboxHead = Dense(512, activation="relu")(bboxHead)



bbox_output = Dense(4, activation='linear', name='bounding_box')(bboxHead)

model_2 = Model(inputs=input_layer, outputs=bbox_output)


In [ ]:
# Compile the model
model_2.compile(optimizer='adam', loss='mean_squared_error')
model_2.summary()

In [ ]:
# Define early stopping callback

# Train the model with early stopping
history_2 = model_2.fit(train_images, train_boxes,
                    validation_data=(val_images, val_boxes),
                    epochs=300, batch_size=8)

In [ ]:
# Save the trained model
model_2.save('/content/drive/MyDrive/trained_bbox_model_3.h5')

In [ ]:
import matplotlib.pyplot as plt

# Get the training and validation loss values from the history object
train_loss = history_2.history['loss']
val_loss = history_2.history['val_loss']

# Get the epoch with the best and worst training loss
best_train_epoch = train_loss.index(min(train_loss)) + 1
worst_train_epoch = train_loss.index(max(train_loss)) + 1

# Get the epoch with the best and worst validation loss
best_val_epoch = val_loss.index(min(val_loss)) + 1
worst_val_epoch = val_loss.index(max(val_loss)) + 1

# Plot the training and validation losses
epochs = range(1, len(train_loss) + 1)
plt.plot(epochs, train_loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot the best and worst values
plt.plot(best_train_epoch, min(train_loss), 'go', label='Best Training Loss')
plt.plot(worst_train_epoch, max(train_loss), 'ro', label='Worst Training Loss')
plt.plot(best_val_epoch, min(val_loss), 'g^', label='Best Validation Loss')
plt.plot(worst_val_epoch, max(val_loss), 'r^', label='Worst Validation Loss')

plt.annotate(f'({best_train_epoch}, {min(train_loss):.4f})',
             xy=(best_train_epoch, min(train_loss)),
             xytext=(best_train_epoch, min(train_loss) + 0.5),
             arrowprops=dict(facecolor='green', arrowstyle='->'))

plt.annotate(f'({worst_train_epoch}, {max(train_loss):.4f})',
             xy=(worst_train_epoch, max(train_loss)),
             xytext=(worst_train_epoch, max(train_loss) + 0.5),
             arrowprops=dict(facecolor='red', arrowstyle='->'))

plt.annotate(f'({best_val_epoch}, {min(val_loss):.4f})',
             xy=(best_val_epoch, min(val_loss)),
             xytext=(best_val_epoch, min(val_loss) + 0.5),
             arrowprops=dict(facecolor='green', arrowstyle='->'))

plt.annotate(f'({worst_val_epoch}, {max(val_loss):.4f})',
             xy=(worst_val_epoch, max(val_loss)),
             xytext=(worst_val_epoch, max(val_loss) + 0.5),
             arrowprops=dict(facecolor='red', arrowstyle='->'))

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
# Perform predictions
predicted_boxes = model_2.predict(test_images)

# Calculate IoU scores for each image
iou_scores = []
for pred_box, true_box in zip(predicted_boxes, test_boxes):
    iou_score = calculate_iou(pred_box, true_box)
    iou_scores.append(iou_score)

# Multiply bounding boxes
multiplied_boxes = np.multiply(predicted_boxes, test_boxes)

# Denormalize bounding boxes for plotting
def denormalize_boxes(boxes, image_shape):
    height, width = image_shape[:2]
    boxes = np.array(boxes)
    boxes[:, [0, 2]] *= width
    boxes[:, [1, 3]] *= height
    return boxes

# Plot images with original and predicted bounding boxes
def plot_image_with_boxes(image, true_boxes, pred_boxes, iou_score, title):
    # Create figure and axes
    fig, ax = plt.subplots(1)
    ax.imshow(image)

    # Plot original bounding boxes
    for box in true_boxes:
        xmin, ymin, xmax, ymax = box
        rect = Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, fill=False, edgecolor='r', linewidth=2)
        ax.add_patch(rect)

    # Plot predicted bounding boxes
    for box in pred_boxes:
        xmin, ymin, xmax, ymax = box
        rect = Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, fill=False, edgecolor='b', linewidth=2)
        ax.add_patch(rect)

    # Set title and IoU text
    ax.set_title(f"{title}\nIoU: {iou_score:.2f}")

    # Show the plot
    plt.show()

# Iterate over each image and plot with bounding boxes and IoU
mean_iou = 0.0
for i in range(len(test_images)):
    # Denormalize bounding boxes
    denorm_test_boxes = denormalize_boxes([test_boxes[i]], test_images[i].shape)
    denorm_predicted_boxes = denormalize_boxes([predicted_boxes[i]], test_images[i].shape)

    # Calculate IoU score
    iou_score = iou_scores[i]
    mean_iou += iou_score

    # Plot the image with original and predicted bounding boxes
    plot_image_with_boxes(test_images[i], denorm_test_boxes, denorm_predicted_boxes, iou_score, "Original and Predicted Bounding Boxes")

# Calculate the mean IoU score
mean_iou /= len(test_images)
print("Mean IoU Score:", mean_iou)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Perform label encoding on the target labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
val_labels_encoded = label_encoder.transform(val_labels)

# Convert the encoded labels to one-hot encoding
train_labels_onehot = to_categorical(train_labels_encoded, num_classes=101)
val_labels_onehot = to_categorical(val_labels_encoded, num_classes=101)


# Model architecture
input_layer = Input(shape=(224, 224, 3))
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=input_layer)

flatten = Flatten()(base_model.output)

bboxHead = Dense(1024, activation="relu")(flatten)

bboxHead = Dense(512, activation="relu")(bboxHead)


bbox_output = Dense(4, activation='linear', name='bounding_box')(bboxHead)


softmaxHead = Dense(1024, activation="relu")(flatten)
softmaxHead = Dropout(0.5)(softmaxHead)


softmaxHead = Dense(512, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)

softmaxHead = Dense(256, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)

class_output = Dense(101, activation='softmax', name='class_output')(softmaxHead)

model = Model(inputs=input_layer, outputs=[class_output, bbox_output])

# Loss function
losses = {
    'class_output': 'categorical_crossentropy',
    'bounding_box': 'mean_squared_error'
}

# Compile the model
model.compile(optimizer='adam', loss=losses)


model.summary()